#Link to the draft:

https://docs.google.com/document/d/1BaUYUjYpS7lKsZLL5jVfBHLOrwUrEfoj_3G3X9ablDc/edit#heading=h.76bewmjofvoz


###Roadmap

*   We can start from Henny's notebook below
*   To measure the distance between the three genres, let's use the 11 features employed by Botond and Szemes 2022 (cf. https://github.com/SzemesBotond/drama_cluster_genre/blob/main/drama_cluster_genre.R)
1. Correlation matrix to exclude redundant measures + discussion of the most distinctive measures
3. Visualisation trough PCA/t-sne, filtered by corpus
4. filtering out last act and seeing what it happens (cf. https://github.com/SzemesBotond/drama_cluster_genre/blob/main/last_act_differences.R)

* Contrastive exploration of operas in DraCor:
- relative frequence of `<div type="prologue">`
- drama change rate
- computing ration sp/stage directions
- intricacy of plot?





##Initialisation
### Load libraries

In [ ]:
# if libraries are not installed, remove the hash from the line starting with '!'
# if you want to reproduce an analysis you can add the version number like this:
# requests==2.25.1 pandas==1.2.3 matplotlib==3.3.4
#! pip install requests pandas matplotlib

In [ ]:
!pip install pydracor

In [ ]:
import pydracor

In [ ]:
import math
from datetime import datetime

import requests
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import os

if not os.path.exists("output-images"):
    os.mkdir("output-images")

In [ ]:
import numpy as np

In [ ]:
import seaborn as sns

In [ ]:
import networkx as nx

In [ ]:
from scipy import stats

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
!pip install wikidata

In [ ]:
from wikidata.client import Client as wikiclient

In [ ]:
import json

#### Get version information for reproducibility

In [ ]:
pip freeze | grep "matplotlib\|pandas\|requests"

#### Get current date for version information of corpus and API

In [ ]:
print(datetime.now())

# I. Data preparation

## 1. Downloading corpora metadata from DraCor

In [ ]:
import requests
from urllib3.exceptions import InsecureRequestWarning
 
# Suppress only the single warning from urllib3 needed.
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

In [ ]:
# switched off due to SSL issues, using requests instead
#ger_corpus = pydracor.Corpus('ger')

In [ ]:
gerdata = requests.get('https://dracor.org/api/corpora/ger/metadata', verify=False)

In [ ]:
type(gerdata.text)

In [ ]:
german_data = pd.DataFrame(json.loads(gerdata.text))

In [ ]:
german_data

In [ ]:
#german_data = ger_corpus.metadata()

In [ ]:
#german_data = pd.DataFrame(german_data)

In [ ]:
german_data.columns

In [ ]:
col_renamer = {'normalizedGenre':'normalized_genre', 
 'firstAuthor':'first_author',
 'numOfSegments':'num_of_segments',
 'numOfSpeakers':'num_of_speakers', 
 'numOfPersonGroups':'num_of_person_groups',
 'wordCountSp':'word_count_sp',
 'wordCountStage':'word_count_stage',
 'yearNormalized':'year_normalized',
 'averageDegree':'average_degree',
 'averageClustering':'average_clustering', 
 'maxDegree':'max_degree',
 'numConnectedComponents':'num_connected_components', 
 'averagePathLength':'average_path_length'}

In [ ]:
german_data = german_data.rename(columns=col_renamer)

In [ ]:
german_data.info()

Initial number of explicitly marked libretti

In [ ]:
german_data[german_data['libretto'] == True].shape[0]

Are there any libretti which also have genre marked? Look at the ambiguous German plays:

In [ ]:
german_data[~(german_data['normalized_genre'].isna()) & (german_data['libretto'] == True)]

None..

Now same for the French

In [ ]:
#fre_corpus = pydracor.Corpus('fre')

In [ ]:
#french_data = fre_corpus.metadata()

In [ ]:
fredata = requests.get('https://dracor.org/api/corpora/fre/metadata', verify=False)

In [ ]:
french_data = json.loads(fredata.text)

In [ ]:
french_data = pd.DataFrame(french_data)

In [ ]:
french_data.columns

In [ ]:
french_data = french_data.rename(columns=col_renamer)

Initial number of explicitly marked libretti

In [ ]:
french_data[french_data['libretto'] == True].shape[0]

Are there any libretti which also have genre marked? Look at the ambiguous French plays:

In [ ]:
french_data[~(french_data['normalized_genre'].isna()) & (french_data['libretto'] == True)]

In [ ]:
noamb = french_data[~(french_data['normalized_genre'].isna()) & (french_data['libretto'] == True)].shape[0]

In [ ]:
print(f'There are {noamb} ambigous french plays')

### Preliminary stats: Libretti vs traditional Genres in the unencriched off-the-shelf corpora 

In [ ]:
sns.set(rc={'figure.figsize':(10, 5.8)})

In [ ]:
datatoplot = german_data.apply(lambda x: x['normalized_genre'] if x['libretto'] == False 
                  else 'Libretto', axis=1).value_counts()

ax = sns.barplot(x=datatoplot.index, y=datatoplot, palette=['darkblue', 'crimson', 'orange', 'green'])
ax.set(title='German Libretto or Genre initital')

for step, index in enumerate(datatoplot.index):
    ax.text(step, datatoplot.loc[index]+1, str(datatoplot.loc[index]), color='black', ha='center')

In [ ]:
datatoplot = french_data.apply(lambda x: x['normalized_genre'] if x['libretto'] == False 
                  else 'Libretto', axis=1).value_counts()

ax = sns.barplot(x=datatoplot.index, y=datatoplot, palette=['darkblue', 'crimson',  'green', 'orange'])
ax.set(title='French Libretto or Genre initital')
for step, index in enumerate(datatoplot.index):
    ax.text(step, datatoplot.loc[index]+2, str(datatoplot.loc[index]), color='black', ha='center')

## 2. Enrichment 

## Uploading libretto corpus from Github

Downloading curated libretti corpus from Github repository


In [ ]:
corpus_libretti_curated = pd.read_csv('https://github.com/DanilSko/opera/raw/main/data/curated_libretti.csv')

In [ ]:
corpus_libretti_curated.columns

In [ ]:
corpus_libretti_curated.describe()

In [ ]:
## corpus_libretti_curated = pd.read_csv('https://github.com/lucagiovannini7/computational-opera-ccls2023/raw/main/opera_dracor_corpus.csv')

## 3. Preprocessing

### Feature selection (new, on the whole data)

In [ ]:
## all features available:
german_data.columns

In [ ]:
len(german_data.columns)

In [ ]:
## features we selected manually as the meaningful ones (dropping year and such stuff)
list_features_pyd = ['num_of_segments', 'num_of_speakers', 'num_of_person_groups',
                      'word_count_sp', 'word_count_stage', 'average_degree', 'density', 'average_clustering',
                      'max_degree', 'num_connected_components', 'diameter', 'average_path_length']

In [ ]:
len(list_features_pyd)

Now we go on to check which of these features correlate with each other:

NEW 31.12: list of network features only:

In [ ]:
network_features = ['num_of_segments', 'num_of_speakers', 'average_degree', 
                     'density', 'average_clustering','max_degree', 'num_connected_components', 
                     'diameter', 'average_path_length']

### Correlation matrices separate for languages

In [ ]:
feature_correlations = german_data[list_features_pyd].corr()

In [ ]:
fig, ax = plt.subplots(figsize=(12,10)) 
sns.heatmap(feature_correlations, annot=True, cmap = 'coolwarm')
plt.savefig('correlation_matrix.png', dpi=300)

In [ ]:
french_data.columns

In [ ]:
list_features_pyd

In [ ]:
feature_correlations = french_data[list_features_pyd].corr()

In [ ]:
feature_correlations

In [ ]:
german_data['num_of_person_groups'].value_counts()

In [ ]:
french_data['num_of_person_groups'].value_counts()

In [ ]:
#feature_correlations

In [ ]:
fig, ax = plt.subplots(figsize=(12,8)) 
sns.heatmap(feature_correlations, annot=True, cmap = 'coolwarm')
plt.savefig('correlation_matrix_fre.png', dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8)) 
sns.heatmap(feature_correlations.drop('num_of_person_groups').drop('num_of_person_groups', axis=1), annot=True, cmap = 'coolwarm')
plt.savefig('correlation_matrix_fre.png', dpi=300)

### Getting highly correlated features as list on formal grounds (above 0.75 or below -0.75 correlation)

### German

In [ ]:
def get_higlhy_correcated_feature_pairs(series):
    correlated = list(series[series == True].index)
    name = series.name
    try:
        correlated.remove(name)
    except:
        pass
    if len(correlated) > 0:
        for cf in correlated:
            print(f'**{name}** is highly correlated with **{cf}**')


In [ ]:
feature_correlations = german_data[list_features_pyd].corr()
colnames  = feature_correlations.columns
hiposcorr = feature_correlations > 0.75
hinegcorr = feature_correlations < -0.75
hicorr = pd.DataFrame(np.nansum([x.values for x in [hiposcorr, hinegcorr]], axis=0))
hicorr.columns = colnames
hicorr.index = colnames
hicorr.apply(get_higlhy_correcated_feature_pairs)

#### Plotting a network of highly correlated features

In [ ]:
def plot_corr_graph(hicorrdf):
    graph = nx.from_pandas_adjacency(hicorrdf)
    graph.remove_edges_from(nx.selfloop_edges(graph))
    plt.subplots(figsize=(14,12)) 
    nx.draw(graph, 
        with_labels=True, 
        pos=nx.spring_layout(graph))

In [ ]:
plot_corr_graph(hicorr)

#### French

In [ ]:
feature_correlations = french_data[list_features_pyd].corr()

In [ ]:
colnames  = feature_correlations.columns
hiposcorr = feature_correlations > 0.75
hinegcorr = feature_correlations < -0.75
hicorr = pd.DataFrame(np.nansum([x.values for x in [hiposcorr, hinegcorr]], axis=0))
hicorr.columns = colnames
hicorr.index = colnames

In [ ]:
hicorr.apply(get_higlhy_correcated_feature_pairs)

#### Plotting a network of highly correlated features

In [ ]:
plot_corr_graph(hicorr)

In [ ]:
#hicorr.apply(lambda x: print(x.name, list(x[x == True].index))) 

### Drop correlated features

In [ ]:
german_features_to_drop = ['average_path_length', 
                           'diameter',  
                           'max_degree', 
                           'num_connected_components']

In [ ]:
french_features_to_drop = ['num_of_segments', 
                           'average_path_length', 
                           'max_degree']

In [ ]:
#german_data = german_data.drop(german_features_to_drop, axis=1)

In [ ]:
#french_data = french_data.drop(french_features_to_drop, axis=1)

In [ ]:
german_features = [feature for feature in list_features_pyd if feature not in german_features_to_drop]

In [ ]:
french_features = [feature for feature in list_features_pyd if feature not in french_features_to_drop]

In [ ]:
#metadata_df = metadata_df.drop(['diameter', 'max_degree'], axis=1)

In [ ]:
# list_features_pyd.remove('diameter')

In [ ]:
# list_features_pyd.remove('max_degree')

In [ ]:
# joining the two dataframes

# replace rows in german_data / french_data with rows from corpus_libretti_curated

In [ ]:
german_data['normalized_genre'].unique()

## Dealing with unlabelled plays through Wikidata scraping 

### German wikidata enrichment

In [ ]:
client = wikiclient()

In [ ]:
def get_genre_from_wikidata(wiki_id):
    if wiki_id == 'No wikidata':
        return None
    entity = client.get(wiki_id, load=True)
    try:
        genre_prop = client.get('P136')
        genre = entity[genre_prop]
        return genre.id
    except:
        return 'No genre on wikidata'

In [ ]:
def get_wikidata_id(playid):
    play = pydracor.Play(playid)
    return play.wikidata_id

In [ ]:
def get_label_and_description_from_wikidata(wiki_id):
    try:
        if 'Q' in wiki_id and wiki_id != 'No wikidata':
            entity = client.get(wiki_id, load=True)
            #return entity.label
            return (entity.label, entity.description)
        #return 
    except:
        pass
     #   return 'No genre on wikidata'

In [ ]:
!wget https://raw.githubusercontent.com/DanilSko/opera/main/data/aux/gerdracor2wikidata.json

In [ ]:
with open('gerdracor2wikidata.json') as jsonmapping:
    dracor2wikidata = json.load(jsonmapping)

In [ ]:
german_data['wikidata'] = german_data['id'].map(dracor2wikidata)

In [ ]:
test = german_data[:4]

In [ ]:
test['genre_from_wikidata'] = test['wikidata'].map(get_genre_from_wikidata).where(test['normalized_genre'].isna(), None)

In [ ]:
test[['name', 'genre_from_wikidata']]

Wikidata querying takes time, so here we'll upload wikidata genre IDs from json saved the previous time instead:

In [ ]:
#pd.read_json('https://raw.githubusercontent.com/DanilSko/opera/main/genre_wikidata_german.json')

In [ ]:
from_json = True

path_to_json = 'https://raw.githubusercontent.com/DanilSko/opera/main/data/aux/genre_wikidata_german.json'

if from_json:
    saved_wikidata_german = pd.read_json(path_to_json, 
                                         typ='series')
    german_data['genre_from_wikidata'] = saved_wikidata_german
else:
    german_data['genre_from_wikidata'] = german_data['wikidata'].map(get_genre_from_wikidata).where(german_data['normalized_genre'].isna(), None)

In [ ]:
german_data[german_data['genre_from_wikidata'].fillna('None').str.contains('Q')][['title', 'normalized_genre']]

In [ ]:
german_data['genre_from_wikidata'].unique()

In [ ]:
german_data[german_data['genre_from_wikidata'] == 'Q1311570'][['name', 'title', 'subtitle', 'genre_from_wikidata']]

In [ ]:
german_data.index

In [ ]:
german_data['genre_from_wikidata'].value_counts()

In [ ]:
german_data['genre_from_wikidata'].to_json('genre_wikidata_german.json', indent=2)

In [ ]:
#used once
#german_data['genre_from_wikidata'].to_json('genre_wikidata_german.json')

In [ ]:
#used once
#french_data['genre_from_wikidata'].to_json('genre_wikidata_french.json')

In [ ]:
german_data['wikidata descr'] = german_data['genre_from_wikidata'].apply(get_label_and_description_from_wikidata)

In [ ]:
wikidata_slice = german_data[(german_data['genre_from_wikidata'].fillna('').str.contains('Q')) & 
            (german_data['libretto'] == False)]

In [ ]:
wikidata_slice

In [ ]:
wikigenres = wikidata_slice['genre_from_wikidata'].unique()

In [ ]:
wikigenres

In [ ]:
for genre in wikigenres:
    label, descr = get_label_and_description_from_wikidata(genre)
    print(genre, '\t', label, '\t', descr)

### French wikidata enrichment

In [ ]:
!wget https://raw.githubusercontent.com/DanilSko/opera/main/data/aux/fredracor2wikidata.json

In [ ]:
with open('fredracor2wikidata.json') as jsonmapping:
    dracor2wikidata = json.load(jsonmapping)

In [ ]:
french_data['wikidata'] = french_data['id'].map(dracor2wikidata)

In [ ]:
french_data['wikidata'].value_counts()

In [ ]:
french_data['wikidata'] = french_data['wikidata'].fillna('No wikidata')

In [ ]:
french_data['wikidata']

Updating wikidata genre codes from saved JSON to spare 6 minutes of wikidata queriyng every time

In [ ]:
from_json = True
path_to_json = 'https://raw.githubusercontent.com/DanilSko/opera/main/data/aux/genre_wikidata_french.json'

if from_json:
    saved_wikidata_french = pd.read_json(path_to_json, 
                                         typ='series')
    #saved_wikidata_french.columns = ['genre_from_wikidata']
    french_data['genre_from_wikidata'] = saved_wikidata_french
else:
    french_data['genre_from_wikidata'] = french_data['wikidata'].map(get_genre_from_wikidata).where(french_data['normalized_genre'].isna(), None)
#french_data['genre_from_wikidata'] = french_data['wikidata'].map(get_genre_from_wikidata).where(french_data['normalized_genre'].isna(), None)

In [ ]:
french_data['genre_from_wikidata'].unique()

In [ ]:
french_data['genre_from_wikidata'].value_counts()

In [ ]:
french_data[french_data['genre_from_wikidata'] == 'Q781470'][['name', 'title', 'subtitle', 'genre_from_wikidata']]

In [ ]:
french_data['genre_from_wikidata'].unique()

In [ ]:
french_data['normalized_genre']

In [ ]:
french_data['wikidata descr'] = french_data['genre_from_wikidata'].apply(get_label_and_description_from_wikidata)

In [ ]:
wikidata_slice = french_data[(french_data['genre_from_wikidata'].fillna('').str.contains('Q')) & 
            (french_data['libretto'] == False)]

In [ ]:
wikidata_slice

In [ ]:
wikigenres = wikidata_slice['genre_from_wikidata'].unique()

In [ ]:
wikigenres

In [ ]:
for genre in wikigenres:
    label, descr = get_label_and_description_from_wikidata(genre)
    print(genre, '\t', label, '\t', descr)

### Filling the normalized genre column

#### 1. Uploading the manuallly created mapper table (from wikidata genres to generalized dracor normalised genres:

In [ ]:
!wget https://raw.githubusercontent.com/DanilSko/opera/main/data/aux/wikidata_genres_mapping.csv

In [ ]:
wd_genre_mapping = pd.read_csv('wikidata_genres_mapping.csv')

In [ ]:
wd_genre_mapping.columns

In [ ]:
norm_genres = wd_genre_mapping["Normalized genre"]

In [ ]:
wd_genre_mapping["Wikidata ID"] = wd_genre_mapping["Wikidata ID"].str.strip()

In [ ]:
wd_genre_ids = wd_genre_mapping["Wikidata ID"]

In [ ]:
genre_mapper = dict(zip(wd_genre_ids, norm_genres))

In [ ]:
genre_mapper

In [ ]:
def get_normalized_genre_for_wd(wiki_genre_id):
    if wiki_genre_id in genre_mapper:
        return genre_mapper[wiki_genre_id]
    return None

#### 2. Mapping the French ones:

In [ ]:
french_data['genre_from_wikidata'].unique()

In [ ]:
french_data['normalized_genre_from_wd'] = french_data['genre_from_wikidata'].apply(get_normalized_genre_for_wd)

In [ ]:
french_data['normalized_genre_from_wd'].unique()

In [ ]:
french_data.shape

In [ ]:
french_data['normalized_genre'].value_counts()

In [ ]:
french_data['normalized_genre'] = french_data['normalized_genre'].fillna(french_data['normalized_genre_from_wd'])

In [ ]:
french_data['normalized_genre'].value_counts()

#### 3. Mapping the German ones:


In [ ]:
german_data['normalized_genre_from_wd'] = german_data['genre_from_wikidata'].apply(get_normalized_genre_for_wd)

In [ ]:
german_data['normalized_genre_from_wd'].unique()

In [ ]:
german_data[german_data['normalized_genre_from_wd'] == 'Libretto (attributed)'][['name', 'genre_from_wikidata']]

In [ ]:
german_data[~(german_data['genre_from_wikidata'].isna()) & 
            (german_data['genre_from_wikidata'] != 'No genre on wikidata') &
            (german_data['normalized_genre'].isna())][['first_author', 
                                                       'title', 
                                                       'normalized_genre', 
                                                       'genre_from_wikidata',
                                                       'normalized_genre_from_wd']]

In [ ]:
german_data['normalized_genre'].value_counts()

In [ ]:
german_data['normalized_genre'] = german_data['normalized_genre'].fillna(german_data['normalized_genre_from_wd'])

In [ ]:
german_data['normalized_genre'].value_counts()

In [ ]:
german_data[german_data['normalized_genre'] == 'Libretto (attributed)']

### Adding libretti information marked by Luca

In [ ]:
libretti_ids = corpus_libretti_curated['id']

In [ ]:
libretti_ids

In [ ]:
#metadata_df['is_ger'] = metadata_df['id'].str.contains('ger')

In [ ]:
#metadata_df[metadata_df['is_ger']].shape[0]

In [ ]:
german_data['is_real_libretto'] = german_data['id'].isin(libretti_ids)

In [ ]:
french_data['is_real_libretto'] = french_data['id'].isin(libretti_ids)

In [ ]:
# how many libretti are there in the german part?
german_data['is_real_libretto'].sum()

In [ ]:
# how many libretti are there in the french part?
french_data['is_real_libretto'].sum()

Adding to 'is_real_libretto' the ones we got from wikidata

In [ ]:
indices = german_data[german_data['normalized_genre'] == 'Libretto (attributed)'].index

In [ ]:
indices

In [ ]:
german_data.loc[indices,:]

In [ ]:
german_data.loc[indices,'is_real_libretto'] = True

In [ ]:
german_data['is_real_libretto'].sum()

same with the french

In [ ]:
indices = french_data[french_data['normalized_genre'] == 'Libretto (attributed)'].index

In [ ]:
indices

In [ ]:
french_data.loc[indices,:]

In [ ]:
french_data.loc[indices,'is_real_libretto'] = True

In [ ]:
# how many libretti are there in the french part?
french_data['is_real_libretto'].sum()

### making a single column with 'Libretto or Genre' in it

#### 1. German:

In [ ]:
german_data['libretto_or_genre'] = german_data.apply(lambda x: 
                                                     x['normalized_genre'] 
                                                     if x['is_real_libretto'] == False 
                                                     else 'Libretto', axis=1)

In [ ]:
german_data['libretto_or_genre'].value_counts()

In [ ]:
german_data['libretto_or_genre'] = german_data['libretto_or_genre'].fillna('Other')

In [ ]:
german_data['libretto_or_genre'].value_counts()

In [ ]:
german_data['libretto_or_genre'].value_counts().plot.bar()

In [ ]:
german_data['libretto_or_genre'] = german_data['libretto_or_genre'].replace('Libretto (attributed)', 
                                                                                                          'Libretto')

#### 2. French:

In [ ]:
french_data['libretto_or_genre'] = french_data.apply(lambda x: 
                                                     x['normalized_genre'] 
                                                     if x['is_real_libretto'] == False 
                                                     else 'Libretto', axis=1)

In [ ]:
french_data['libretto_or_genre'].value_counts()

In [ ]:
french_data['libretto_or_genre'] = french_data['libretto_or_genre'].fillna('Other')

In [ ]:
french_data['libretto_or_genre'].value_counts()

In [ ]:
french_data['libretto_or_genre'].value_counts().plot.bar()

### Libretti apartheid: separating libretti Luca marked himself from the 'authority' dracor&wikidata libretti

#### German

In [ ]:
german_data['genre_with_putative_libretto'] = german_data.apply(lambda x: 'Libretto (attributed)' if 
                  x['is_real_libretto'] is True and x['libretto'] is False
                  else x['libretto_or_genre'], axis=1)

In [ ]:
german_data['genre_with_putative_libretto'].value_counts()

In [ ]:
german_data['genre_with_putative_libretto'] = german_data['genre_with_putative_libretto'].replace('Libretto', 'Libretto (DraCor)')

In [ ]:
german_data['genre_with_putative_libretto'].value_counts()

In [ ]:
x = german_data[german_data['genre_with_putative_libretto'] !='Other'].shape[0]
y = german_data.shape[0]

In [ ]:
x/y

### French

In [ ]:
french_data['genre_with_putative_libretto'] = french_data.apply(lambda x: 'Libretto (attributed)' if 
                  x['is_real_libretto'] is True and x['libretto'] is False
                  else x['libretto_or_genre'], axis=1)

In [ ]:
french_data['genre_with_putative_libretto'].value_counts()

In [ ]:
french_data['genre_with_putative_libretto'] = french_data['genre_with_putative_libretto'].replace('Libretto', 'Libretto (DraCor)')

In [ ]:
french_data['genre_with_putative_libretto'].value_counts()

In [ ]:
french_data[french_data['genre_with_putative_libretto'] !='Other'].shape

In [ ]:
1178/1560

#### Adding colors according to genres

In [ ]:
genres = list(german_data['genre_with_putative_libretto'].unique())

In [ ]:
len(genres)

In [ ]:
genres

In [ ]:
colors = ['white', 'green', 'blue', 'orange', 'yellow', 'red']

In [ ]:
genre_color_mapping = zip(genres, colors)

In [ ]:
genre_color_mapping = dict(genre_color_mapping)

In [ ]:
genre_color_mapping

In [ ]:
german_data['color'] = german_data['genre_with_putative_libretto'].apply(lambda x: 
                                                              genre_color_mapping[x])

In [ ]:
french_data['color'] = french_data['genre_with_putative_libretto'].apply(lambda x: 
                                                              genre_color_mapping[x])

In [ ]:
#german_data.loc[:, features].min()

In [ ]:
#german_data.loc[:, features].apply(np.log)

In [ ]:
#def remove_outliers(pca_loadings):



### Removing 'Other'

In [ ]:
filtered_german_data = german_data[german_data['libretto_or_genre'] != 'Other']
filtered_french_data = french_data[french_data['libretto_or_genre'] != 'Other']

In [ ]:
## removing 3 french plays with NaNs
to_remove = filtered_french_data[(filtered_french_data['average_degree'].isna()) 
& (filtered_french_data['density'].isna())].index

In [ ]:
filtered_french_data = filtered_french_data.drop(to_remove)

### Shorter names for dfs

In [ ]:
gd = filtered_german_data

In [ ]:
fd = filtered_french_data

### Analytics: how much more after enrichment

In [ ]:
gd['genre_with_putative_libretto'].value_counts()

In [ ]:
newly_added_sum = gd['genre_with_putative_libretto'].value_counts().loc['Libretto (attributed)']

In [ ]:
datatoplot = gd['libretto_or_genre'].value_counts()

ax = sns.barplot(x=datatoplot.index, y=datatoplot, palette=['darkblue', 'crimson', 'orange', 'green'])
ax.set(title='German Libretto or Genre after Enrichment')

for step, index in enumerate(datatoplot.index):
    string = ''
    if index == 'Libretto':
        string = f' (+{newly_added_sum})'
    ax.text(step, datatoplot.loc[index]+1, str(datatoplot.loc[index]) + string, color='black', ha='center')

In [ ]:
fd['genre_with_putative_libretto'].value_counts()

In [ ]:
newly_added_sum_fr = fd['genre_with_putative_libretto'].value_counts().loc['Libretto (attributed)']

In [ ]:
datatoplot = fd['libretto_or_genre'].value_counts()

ax = sns.barplot(x=datatoplot.index, y=datatoplot, palette=['darkblue', 'crimson', 'orange', 'green'])
ax.set(title='French Libretto or Genre after Enrichment')

for step, index in enumerate(datatoplot.index):
    string = ''
    if index == 'Libretto':
        string = f' (+{newly_added_sum_fr})'
    ax.text(step, datatoplot.loc[index]+1, str(datatoplot.loc[index]) + string, color='black', ha='center')

### Distinguish comic and non-comic libretti

In [ ]:
gd['subtitle'] = gd['subtitle'].fillna('-')

In [ ]:
fd['subtitle'] = fd['subtitle'].fillna('-')

In [ ]:
gd[gd['subtitle'].str.contains('Oper')][['first_author', 'title', 'subtitle']]

In [ ]:
fd[fd['subtitle'].str.contains('Opéra')][['first_author', 'title', 'subtitle']]

In [ ]:
list(gd[gd['is_real_libretto']==True]['subtitle'])

In [ ]:
list(fd[fd['is_real_libretto']==True]['subtitle'])

In [ ]:
import re

In [ ]:
def mark_comic_opera(some_subtitle, lang='de'):
    some_subtitle = some_subtitle.lower()
    comic_regex_fr = re.compile('comique|operette|comédie|comedie|vaudevill|divertissement')
    comic_regex_de = re.compile('komisch|operette|komödie|comedie|parodie|posse')
    if lang=='de':
        regex = comic_regex_de
    else:
        regex = comic_regex_fr
    if re.search(regex, some_subtitle) is not None:
        return 'Comic libretto'
    else:
        return 'Non-comic libretto'

In [ ]:
gd['libretto_subgenre'] = gd['subtitle'].apply(mark_comic_opera)

In [ ]:
gd[gd['is_real_libretto']==False]['libretto_subgenre'] = 'Not libretto'

In [ ]:
gd['libretto_subgenre']

In [ ]:
gd['genre_with_libretto_subgenres'] = gd.apply(lambda x: x['libretto_or_genre'] 
                                               if x['is_real_libretto'] == False
                                               else x['libretto_subgenre'], 
                                               axis=1)

In [ ]:
gd['genre_with_libretto_subgenres'].value_counts()

In [ ]:
gd = gd[gd['genre_with_libretto_subgenres'] != 'Libretto']

In [ ]:
gd['genre_with_libretto_subgenres'].value_counts()

In [ ]:
fd['libretto_subgenre'] = fd['subtitle'].apply(mark_comic_opera, lang='fr')

In [ ]:
fd['genre_with_libretto_subgenres'] = fd.apply(lambda x: x['libretto_or_genre'] 
                                               if x['is_real_libretto'] == False
                                               else x['libretto_subgenre'], 
                                               axis=1)

In [ ]:
fd['genre_with_libretto_subgenres'].value_counts()

### Color 2 for libretti subgenre

In [ ]:
fd.columns

In [ ]:
genres_with_subgenres = list(fd['genre_with_libretto_subgenres'].unique())

In [ ]:
genres_with_subgenres

In [ ]:
colors_sg = ['red', 'blue', 'orange', 'aquamarine', 'green']

In [ ]:
genre_color_mapping_sg = dict(zip(genres_with_subgenres, colors_sg))

In [ ]:
genre_color_mapping_sg

In [ ]:
fd['color_subgenres'] = fd['genre_with_libretto_subgenres'].apply(lambda x: 
                                                              genre_color_mapping_sg[x])

In [ ]:
gd['color_subgenres'] = gd['genre_with_libretto_subgenres'].apply(lambda x: 
                                                              genre_color_mapping_sg[x])

### Checking timeframes

#### German

In [ ]:
gd[gd['genre_with_putative_libretto'].str.contains('Libretto')]['year_normalized'].describe()

In [ ]:
gd[(gd['genre_with_putative_libretto'].str.contains(
    'Libretto')) & (gd['year_normalized'] == 1770)] 

In [ ]:
gd[(gd['genre_with_putative_libretto'].str.contains(
    'Libretto')) & (gd['year_normalized'] == 1920)] 

#### French

In [ ]:
fd[fd['genre_with_putative_libretto'].str.contains('Libretto')]['year_normalized'].describe()

In [ ]:
libretto_years = fd[fd['genre_with_putative_libretto'].str.contains('Libretto')]['year_normalized']

In [ ]:
libretto_years.max() - libretto_years.min() 

In [ ]:
fd[(fd['genre_with_putative_libretto'].str.contains(
    'Libretto')) & (fd['year_normalized'] == 1626)] 

In [ ]:
fd[(fd['genre_with_putative_libretto'].str.contains(
    'Libretto')) & (fd['year_normalized'] == 1889)] 

In [ ]:
libretto_years.hist(bins=30)

In [ ]:
libretto_years.sort_values().to_csv('year_to_see.txt', index=False)

## Creating time slices

#### 1770 - 1819

In [ ]:
german_data_1770_1819 = gd[(gd['year_normalized'] >= 1770) & 
                                      (gd['year_normalized'] <= 1819)]

In [ ]:
german_data_1770_1819['libretto'].value_counts()

In [ ]:
german_data_1770_1819['genre_with_putative_libretto'].str.contains('Libretto').sum()

In [ ]:
german_data_1770_1819['genre_with_putative_libretto'].value_counts()

#### 1820 - 1869

In [ ]:
german_data_1820_1869 = gd[(gd['year_normalized'] >= 1820) & 
                                      (gd['year_normalized'] <= 1869)]

In [ ]:
german_data_1820_1869['genre_with_putative_libretto'].str.contains('Libretto').sum()

#### 1870 - 1920

In [ ]:
german_data_1870_1920 = gd[(gd['year_normalized'] >= 1870) & 
                                      (gd['year_normalized'] <= 1920)]

In [ ]:
german_data_1870_1920['genre_with_putative_libretto'].str.contains('Libretto').sum()

In [ ]:
german_data_1870_1920.columns

In [ ]:
german_data_1870_1920[german_data_1870_1920["genre_with_libretto_subgenres"] == 'Non-comic libretto'][['first_author','title', 'word_count_sp']].sort_values('word_count_sp')

In [ ]:
def slice_and_get_proportion(data, startyear, endyear):
    data = data[(data['year_normalized'] >= startyear) & 
                                      (data['year_normalized'] <= endyear)]
    no_librettti = data['genre_with_putative_libretto'].str.contains('Libretto').sum()
    no_total = data.shape[0]
    no_nonlibretti = no_total - no_librettti
    print(f'Period from {startyear} to {endyear}:')
    print(f'Libretti: {no_librettti}')
    print(f'Other: {no_nonlibretti}')
    return data

In [ ]:
french_data_1620_1669 =  slice_and_get_proportion(fd, 1620, 1669)

In [ ]:
french_data_1670_1719 = slice_and_get_proportion(fd, 1670, 1719)

In [ ]:
french_data_1670_1719[french_data_1670_1719['is_real_libretto']==True].sort_values('word_count_stage', ascending=False)[['first_author', 'title', 'word_count_stage', 'num_of_speakers']]

In [ ]:
gd[['density', 'average_path_length']].sort_values('average_path_length')

In [ ]:
px.scatter(gd, x='density', y='average_path_length', labels='title')

In [ ]:
french_data_1670_1719[french_data_1670_1719['is_real_libretto']==True][['first_author', 'title', 'year_normalized', 'density', 'num_of_speakers']].to_excel('1670_1719_libretti.xls')

In [ ]:
french_data_1720_1769 = slice_and_get_proportion(fd, 1720, 1769)

In [ ]:
french_data_1770_1819 = slice_and_get_proportion(fd, 1770, 1819)

In [ ]:
french_data_1820_1889 = slice_and_get_proportion(fd, 1820, 1889)

## At this point we have our corpus ready for exploration (all genres are mapped)

# II. Data exploration

## 4. Plotting

## Trying multidimensional methods (PCA, t-SNE, LDA, UMAP)

#### Preprocessing: data standardization with standard scaler

In [ ]:
import time

In [ ]:
from sklearn.preprocessing import StandardScaler
## Standardizing the data
german_features_to_process = gd.loc[:, german_features].values
german_standardized_features = StandardScaler().fit_transform(german_features_to_process)

In [ ]:
def standardize(df, feature_list):
    data_to_process = df.loc[:, feature_list].values
    standardized_data_to_process = StandardScaler().fit_transform(data_to_process)
    return standardized_data_to_process

In [ ]:
french_standardized_features = standardize(fd, french_features)

In [ ]:
#len(features_to_process)

In [ ]:
#len(standardized_features)

### Common visualization functions

In [ ]:
import plotly.io as plt_io
import plotly.graph_objects as go
!pip install kaleido #static image generation
import kaleido

In [ ]:
def plot_2d_no_legend(df, component1, component2, output_filename, title):
    
    fig = go.Figure(data=go.Scatter(
        x = component1,
        y = component2,
        mode='markers',
        marker=dict(
            size=20,
            color=df['color_subgenres'], #set color equal to a variable
            colorscale='Rainbow', # one of plotly colorscales
            showscale=False,
            line_width=1
        ),
        text=df['subtitle'],
        #showlegend=True
    ))
    fig.update_layout(margin=dict( l=100,r=100,b=100,t=100),
                      width=2000,height=1200,
                      font=dict(size=18),
                      title=title)                 
    fig.layout.template = 'plotly'
    
    fig.show()
    fig.write_image("output-images/" + output_filename + ".png",scale=2)

In [ ]:
def plot_2d(df, component1, component2, output_filename, title):

    df['comp 1'] = component1
    df['comp 2'] = component2
    pxscatter = px.scatter(df, x='comp 1', y='comp 2', 
                           color_discrete_sequence=list(df['color_subgenres'].unique()), 
                           color='genre_with_libretto_subgenres',
                           hover_data=['subtitle']
                           #text='subtitle'

    )



    #fig = go.Figure(data=go.Scatter(
     #   x = component1,
      #  y = component2,
       # mode='markers',
        #marker=dict(
         #   size=20,
          #  color=df['color_subgenres'], #set color equal to a variable
           # colorscale='Rainbow', # one of plotly colorscales
            #showscale=False,
            #line_width=1
        #),
        #text=df['subtitle'],
        #showlegend=True
    #))
    fig = go.Figure(data = pxscatter)
    fig.update_traces(marker=dict(size=20,
                              line=dict(width=1)),
                                        #color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.update_layout(margin=dict( l=100,r=100,b=100,t=100),
                      width=2000,height=1200,
                      font=dict(size=18),
                      title=title,
                      legend=dict(title="Genre")
                      )                 
    fig.layout.template = 'plotly'
    
    fig.show()
    fig.write_image("output-images/" + output_filename + ".png",scale=2)

In [ ]:
gd.columns

In [ ]:
def plot_2d_no_subgenres(df, component1, component2, output_filename, title):

    df['comp 1'] = component1
    df['comp 2'] = component2
    pxscatter = px.scatter(df, x='comp 1', y='comp 2', 
                           color_discrete_sequence=['red','blue', 'orange', 'green'], 
                           color='libretto_or_genre',
                           hover_data=['subtitle']

    )

    fig = go.Figure(data = pxscatter)
    fig.update_traces(marker=dict(size=20,
                              line=dict(width=1)),
                                        #color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.update_layout(margin=dict( l=100,r=100,b=100,t=100),
                      width=2000,height=1200,
                      font=dict(size=18),
                      title=title,
                      legend=dict(title="Genre")
                      )                 
    fig.layout.template = 'plotly'
    
    fig.show()
    fig.write_image("output-images/" + output_filename + ".png",scale=2)

#### t-SNE

##### German

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
start = time.time()
tsne = TSNE(random_state = 42, 
            n_components=3,
            verbose=0, 
            perplexity=50, 
            n_iter=400).fit_transform(german_standardized_features)
print('Duration: {} seconds'.format(time.time() - start))

In [ ]:
#plot_2d(filtered_german_data, tsne[:, 0], tsne[:, 1])

#### German 1770-1819


In [ ]:
def make_tsne(df, feature_list, perp, output_filename, title):
    standardized_data = standardize(df, feature_list)
    start = time.time()
    tsne = TSNE(random_state = 42, 
            n_components=3,
            verbose=0, 
            perplexity=perp, 
            n_iter=400).fit_transform(standardized_data)
    print('Duration: {} seconds'.format(time.time() - start))
    plot_2d(df, tsne[:, 0], tsne[:, 1], output_filename, title)


In [ ]:
make_tsne(german_data_1770_1819, german_features, 10, "german_data_1770_1819", "german_data_1770_1819")

In [ ]:
make_tsne(german_data_1770_1819, german_features, 50, "german_data_1770_1819-50", "german_data_1770_1819")

In [ ]:
make_tsne(german_data_1820_1869, german_features, 10, "german_data_1820_1869", "german_data_1820_1869")

In [ ]:
make_tsne(german_data_1820_1869, german_features, 50, "german_data_1820_1869-50", "german_data_1820_1869")

In [ ]:
make_tsne(german_data_1870_1920, german_features, 10, "german_data_1820_1869", "german_data_1870_1920")

In [ ]:
make_tsne(german_data_1870_1920, german_features, 50, "german_data_1870_1920-50","german_data_1870_1920")

#### French

In [ ]:
len(french_standardized_features)

In [ ]:
start = time.time()
tsne = TSNE(random_state = 42, 
            n_components=3,
            verbose=0, 
            perplexity=50, 
            n_iter=400).fit_transform(french_standardized_features)
print('Duration: {} seconds'.format(time.time() - start))

In [ ]:
plot_2d(fd, tsne[:, 0], tsne[:, 1], 'french_tsne', 'french_tsne')

In [ ]:
make_tsne(french_data_1620_1669, french_features, 10, "french_data_1620_1669", "french_data_1620_1669")

In [ ]:
make_tsne(french_data_1620_1669, french_features, 50, "french_data_1620_1669-50", "french_data_1620_1669")

In [ ]:
make_tsne(french_data_1670_1719, french_features, 10, "french_data_1670_1719", "french_data_1670_1719")

In [ ]:
make_tsne(french_data_1670_1719, french_features, 50, "french_data_1670_1719-50", "french_data_1670_1719")

In [ ]:
make_tsne(french_data_1720_1769, french_features, 10, "french_data_1720_1769", "french_data_1720_1769")

In [ ]:
make_tsne(french_data_1720_1769, french_features, 50, "french_data_1720_1769-50", "french_data_1720_1769")

In [ ]:
make_tsne(french_data_1770_1819, french_features, 10, "french_data_1770_1819", "french_data_1770_1819")

In [ ]:
make_tsne(french_data_1770_1819, french_features, 50, "french_data_1770_1819-50", "french_data_1770_1819")

In [ ]:
make_tsne(french_data_1820_1889, french_features, 10, "french_data_1820_1889", "french_data_1820_1889")

In [ ]:
make_tsne(french_data_1820_1889, french_features, 50, "french_data_1820_1889-50", "french_data_1820_1889")

In [ ]:
# download all pics as zip
#from google.colab import files
#!zip -r /content/output_images.zip /content/output-images
#files.download('/content/output_images.zip')

### UMAP

In [ ]:
!pip install umap-learn

In [ ]:
import umap

### German

In [ ]:
start = time.time()
reducer = umap.UMAP(random_state=42,n_components=3)
embedding = reducer.fit_transform(german_standardized_features)
print('Duration: {} seconds'.format(time.time() - start))

In [ ]:
plot_2d(gd, reducer.embedding_[:, 0],reducer.embedding_[:, 1], 'umap_german', 'umap_german')

In [ ]:
def make_umap(df, feature_list, title, filename):
    standardized_data = standardize(df, feature_list)
    start = time.time()
    reducer = umap.UMAP(random_state=42,n_components=3)
    embedding = reducer.fit_transform(standardized_data)
    print('Duration: {} seconds'.format(time.time() - start))
    plot_2d(df, reducer.embedding_[:, 0],reducer.embedding_[:, 1], title, filename)


#### German timestamps

In [ ]:
make_umap(german_data_1770_1819, german_features, 'german_data_1770_1819', 'german_data_1770_1819')

In [ ]:
make_umap(german_data_1820_1869, german_features, 'german_data_1820_1869', 'german_data_1820_1869')

In [ ]:
make_umap(german_data_1870_1920, german_features, 'german_data_1870_1920', 'german_data_1870_1920')

### French

In [ ]:
make_umap(fd, french_features, 'french umap', 'french umap')

#### French timestamps

In [ ]:
make_umap(french_data_1620_1669, french_features,'french_data_1620_1669','french_data_1620_1669')

In [ ]:
make_umap(french_data_1670_1719, french_features, 'french_data_1670_1719', 'french_data_1670_1719')

In [ ]:
make_umap(french_data_1720_1769, french_features, 'french_data_1720_1769', 'french_data_1720_1769')

In [ ]:
make_umap(french_data_1770_1819, french_features, 'french_data_1770_1819', 'french_data_1770_1819')

In [ ]:
make_umap(french_data_1820_1889, french_features, 'french_data_1820_1889', 'french_data_1820_1889')

### PCA

### German

In [ ]:
def make_pca(df, feature_list, output_filename, title):
    standardized_data = standardize(df, feature_list)
    start = time.time()
    pca = PCA(n_components=3)
    principalComponents = pca.fit_transform(standardized_data)
    print('Duration: {} seconds'.format(time.time() - start))
    principal = pd.DataFrame(data = principalComponents
                , columns = ['principal component 1', 'principal component 2','principal component 3'])
    plot_2d(df, principalComponents[:, 0],
            principalComponents[:, 1], 
            output_filename, 
            title)

In [ ]:
make_pca(gd, german_features, 'German PCA', 'German PCA')

#### German timestamps

In [ ]:
make_pca(german_data_1770_1819, german_features, 'pcagerman_data_1770_1819', 'pcagerman_data_1770_1819')

In [ ]:
make_pca(german_data_1770_1819, german_features, '1770_1819', '1770_1819')
make_pca(german_data_1770_1819, list_features_pyd, '1770_1819allfeatures', '1770_1819allfeatures') 

In [ ]:
make_pca(german_data_1820_1869, german_features, 'pcagerman_data_1820_1869', 'pcagerman_data_1820_1869')

In [ ]:
make_pca(german_data_1870_1920, german_features, 'pcagerman_data_1870_1920', 'pcagerman_data_1870_1920')

### French

In [ ]:
make_pca(gd, french_features, 'French PCA', 'French PCA')

#### French timestamps

In [ ]:
make_pca(french_data_1620_1669, french_features, 'pcafrench_data_1620_1669', 'pcafrench_data_1620_1669')

In [ ]:
make_pca(french_data_1670_1719, french_features, 'pcafrench_data_1670_1719', 'pcafrench_data_1670_1719')

In [ ]:
make_pca(french_data_1720_1769, french_features, 'pcafrench_data_1720_1769', 'pcafrench_data_1720_1769')

In [ ]:
make_pca(french_data_1770_1819, french_features, 'pcafrench_data_1770_1819', 'pcafrench_data_1770_1819')

In [ ]:
make_pca(french_data_1820_1889, french_features, 'pcafrench_data_1820_1889', 'pcafrench_data_1820_1889')

### Trying only network features (31.12):

#### t-SNE

##### German

In [ ]:
#make_tsne(filtered_german_data, network_features, 10)

#### German 1770-1819


In [ ]:
#make_tsne(german_data_1770_1819, network_features, 10)

In [ ]:
#make_tsne(german_data_1770_1819, network_features, 50)

In [ ]:
#make_tsne(german_data_1820_1869, network_features, 10)

In [ ]:
#make_tsne(german_data_1820_1869, network_features, 50)

In [ ]:
#make_tsne(german_data_1870_1920, network_features, 10)

In [ ]:
#make_tsne(german_data_1870_1920, network_features, 50)

#### French

In [ ]:
#make_tsne(filtered_french_data, network_features, 10)

In [ ]:
#make_tsne(french_data_1620_1669, network_features, 10)

In [ ]:
#make_tsne(french_data_1620_1669, network_features, 50)

In [ ]:
#make_tsne(french_data_1670_1719, network_features, 10)

In [ ]:
#make_tsne(french_data_1670_1719, network_features, 50)

In [ ]:
#make_tsne(french_data_1720_1769, network_features, 10)

In [ ]:
#make_tsne(french_data_1720_1769, network_features, 50)

In [ ]:
#make_tsne(french_data_1770_1819, network_features, 10)

In [ ]:
#make_tsne(french_data_1770_1819, network_features, 50)

In [ ]:
#make_tsne(french_data_1820_1889, network_features, 10)

In [ ]:
#make_tsne(french_data_1820_1889, network_features, 50)

#### UMAP

#### German

In [ ]:
#make_umap(filtered_german_data, network_features)

#### German timestamps

In [ ]:
#make_umap(german_data_1770_1819, network_features)

In [ ]:
#make_umap(german_data_1820_1869, network_features)

In [ ]:
#make_umap(german_data_1870_1920, network_features)

#### French

In [ ]:
#make_umap(filtered_french_data, network_features)

#### French timestamps

In [ ]:
#make_umap(french_data_1620_1669, network_features)

In [ ]:
#make_umap(french_data_1670_1719, network_features)

In [ ]:
#make_umap(french_data_1720_1769, network_features)

In [ ]:
#make_umap(french_data_1770_1819, network_features)

In [ ]:
#make_umap(french_data_1820_1889, network_features)

#### neetowrk PCA

### German

In [ ]:
#make_pca(gd, network_features)

#### German timestamps

In [ ]:
#make_pca(german_data_1770_1819, network_features)

In [ ]:
#make_pca(german_data_1820_1869, network_features)

In [ ]:
#make_pca(german_data_1870_1920, network_features)

### French

In [ ]:
#make_pca(fd, network_features)

#### French timestamps

In [ ]:
#make_pca(french_data_1620_1669, network_features)

In [ ]:
#make_pca(french_data_1670_1719, network_features)

In [ ]:
#make_pca(french_data_1720_1769, network_features)

In [ ]:
#make_pca(french_data_1770_1819, network_features)

In [ ]:
#make_pca(french_data_1820_1889, network_features)

### PCA for German and French stuff together!

In [ ]:
df_both = pd.concat([gd, fd])

In [ ]:
list_features_pyd

In [ ]:
def attach_lang(row):
    #print(some_df.columns)
   #genre = row.loc['genre_with_libretto_subgenres']
    #lang = row.loc['id']
    #lang = some_df['id'].apply(lambda x: x[:3])
    lang_specific_genre = lang + genre
    return lang_specific_genre

In [ ]:
df_both['lang'] = df_both['id'].apply(lambda x: x[:3])

In [ ]:
df_both['lang_genre_with_libretto_subgenres'] = df_both['lang']+' '+df_both['genre_with_libretto_subgenres']

In [ ]:
df_both['lang_genre_with_libretto_subgenres']

In [ ]:
df_both['bicolor'] = df_both['lang'].apply(lambda x: 'blue' if x == 'fre' else 'orange')

In [ ]:
def make_pca_bilingual(df, feature_list, output_filename, title):
    standardized_data = standardize(df, feature_list)
    start = time.time()
    pca = PCA(n_components=3)
    principalComponents = pca.fit_transform(standardized_data)
    print('Duration: {} seconds'.format(time.time() - start))
    principal = pd.DataFrame(data = principalComponents
                , columns = ['principal component 1', 'principal component 2','principal component 3'])
    plot_2d_bilingual(df, principalComponents[:, 0],
            principalComponents[:, 1], 
            output_filename, 
            title)

In [ ]:
def plot_2d_bilingual(df, component1, component2, output_filename, title):

    df['comp 1'] = component1
    df['comp 2'] = component2
    pxscatter = px.scatter(df, x='comp 1', y='comp 2', 
                           color_discrete_sequence=list(df['bicolor'].unique()), # 'lang_color_subgenres'
                           color = 'lang',
                           #color='lang_genre_with_libretto_subgenres',
                           hover_data=['title']
                           #text='subtitle'

    )

    fig = go.Figure(data = pxscatter)
    fig.update_traces(marker=dict(size=12,
                              line=dict(width=1)),
                                        #color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.update_layout(margin=dict( l=100,r=100,b=100,t=100),
                      width=2000,height=1200,
                      font=dict(size=18),
                      title=title,
                      legend=dict(title="Genre")
                      )                 
    fig.layout.template = 'plotly'
    
    fig.show()
    fig.write_image("output-images/" + output_filename + ".png",scale=2)

In [ ]:
make_pca_bilingual(df_both, list_features_pyd, 'PCA both corpora', 'PCA both corpora')

In [ ]:
df_both['color_subgenres'].unique()

In [ ]:
german_recolorer = {'blue':'#6495ED',
                    'red':'#8B0000',
                    'aquamarine':'#00CED1',
                    'green':'#8FBC8F',
                    'orange':'#F0E68C'}

In [ ]:
df_both['lang_color_subgenres'] = df_both.apply(lambda x: x['color_subgenres'] 
                                                if x['lang'] == 'fre' 
                                                else german_recolorer[x['color_subgenres']],
                                                axis=1)

In [ ]:
def make_pca_bilingual_genre_aware(df, feature_list, output_filename, title, centroids=False):
    standardized_data = standardize(df, feature_list)
    start = time.time()
    pca = PCA(n_components=3)
    principalComponents = pca.fit_transform(standardized_data)
    print('Duration: {} seconds'.format(time.time() - start))
    principal = pd.DataFrame(data = principalComponents
                , columns = ['principal component 1', 'principal component 2','principal component 3'])
    if centroids:
        plot_2d_bilingual_genre_aware_centroids(df, principalComponents[:, 0],
            principalComponents[:, 1], 
            output_filename, 
            title)
    else:
        plot_2d_bilingual_genre_aware(df, principalComponents[:, 0],
            principalComponents[:, 1], 
            output_filename, 
            title)    




In [ ]:
def plot_2d_bilingual_genre_aware(df, component1, component2, output_filename, title):

    df['comp 1'] = component1
    df['comp 2'] = component2
    pxscatter = px.scatter(df, x='comp 1', y='comp 2', 
                           color_discrete_sequence=list(df['lang_color_subgenres'].unique()), # 'lang_color_subgenres'
                           #color = 'lang',
                           color='lang_genre_with_libretto_subgenres',
                           hover_data=['title']
                           #text='subtitle'

    )

    fig = go.Figure(data = pxscatter)
    fig.update_traces(marker=dict(size=12,
                              line=dict(width=1)),
                                        #color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.update_layout(margin=dict( l=100,r=100,b=100,t=100),
                      width=2000,height=1200,
                      font=dict(size=18),
                      title=title,
                      legend=dict(title="Genre")
                      )                 
    fig.layout.template = 'plotly'
    
    fig.show()
    fig.write_image("output-images/" + output_filename + ".png",scale=2)

In [ ]:
make_pca_bilingual_genre_aware(df_both, list_features_pyd, 'PCA both corpora', 'PCA both corpora')

In [ ]:
df_libretti_only = df_both[df_both['is_real_libretto']==True]

In [ ]:
df_libretti_only['lang'].value_counts()

In [ ]:
onlylibr_recolorer = {'aquamarine':'aquamarine',
                    '#F0E68C':'#FF8C00',
                    '#00CED1':'#FFD700',
                    'orange':'#00CED1'}

In [ ]:
df_libretti_only['lang_color_subgenres'] = df_libretti_only['lang_color_subgenres'].apply(lambda x: onlylibr_recolorer[x])

In [ ]:
make_pca_bilingual_genre_aware(df_libretti_only, list_features_pyd, 'PCA both corpora libretti only', 'PCA both corpora libretti only')

In [ ]:
def make_tsne_bilingual_genre_aware(df, feature_list, perp, output_filename, title):
    standardized_data = standardize(df, feature_list)
    start = time.time()
    tsne = TSNE(random_state = 42, 
            n_components=3,
            verbose=0, 
            perplexity=perp, 
            n_iter=400).fit_transform(standardized_data)
    print('Duration: {} seconds'.format(time.time() - start))
    plot_2d_bilingual_genre_aware(df, tsne[:, 0], tsne[:, 1], output_filename, title)

In [ ]:
make_tsne_bilingual_genre_aware(df_libretti_only, list_features_pyd, 25, 't-SNE both corpora libretti only', 't-SNE both corpora libretti only')

In [ ]:
def plot_2d_bilingual_genre_aware_centroids(df, component1, component2, output_filename, title):

    df['comp 1'] = component1
    df['comp 2'] = component2
    pxscatter = px.scatter(df, x='comp 1', y='comp 2', 
                           color_discrete_sequence=list(df['lang_color_subgenres'].unique()), # 'lang_color_subgenres'
                           #color = 'lang',
                           color='lang_genre_with_libretto_subgenres',
                           hover_data=['title']
                           #text='subtitle'

    )
        


    fig = go.Figure(data = pxscatter)
    fig.update_traces(marker=dict(size=20,
                              line=dict(width=1)),
                                        #color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.update_layout(margin=dict( l=100,r=100,b=100,t=100),
                      width=2000,height=1200,
                      font=dict(size=18),
                      title=title,
                      legend=dict(title="Genre")
                      )                 
    fig.layout.template = 'plotly'


    centroids = df.groupby('lang_genre_with_libretto_subgenres').mean()[['comp 1', 'comp 2']]
    fig.add_trace(
        go.Scatter(
                mode='markers',
                x=centroids['comp 1'],
                y=centroids['comp 2'],
                text=centroids.index,
                marker=dict(
                    color= df.groupby('lang_genre_with_libretto_subgenres').first()['lang_color_subgenres'], #df['lang_color_subgenres'].unique(),
                    size=30,
                    symbol="x",
                    line=dict(
                        color='black',
                        width=2
                    )
                ),
                showlegend=False
            )
    )

    
    
    fig.show()
    fig.write_image("output-images/" + output_filename + ".png",scale=2)

In [ ]:
make_pca_bilingual_genre_aware(df_libretti_only, list_features_pyd, 
                                'PCA both corpora libretti only', 
                                'PCA both corpora libretti only',
                               centroids=True)

In [ ]:
list(df_libretti_only.groupby('lang_genre_with_libretto_subgenres').first()['lang_color_subgenres'])

### LDA 

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
def make_lda(df, features, filename='some LDA', title='some LDA'):
    x = df.loc[:, features].values
    x = StandardScaler().fit_transform(x)
    """LDA is supervised; 
    it tries to differentiate classes; 
    so we need to give it classes; in our case they are libretto_or_genre: 
    here two types of libretti are not differentiated. 
    But we color them differently in the viz"""
    X_LDA_fitted = LDA(n_components=2).fit(x, df['libretto_or_genre'])
    X_LDA = X_LDA_fitted.transform(x)
    print(len(X_LDA))
    scalings = X_LDA_fitted.scalings_
    plot_2d(df, X_LDA[:, 0],X_LDA[:, 1], filename, title)


In [ ]:
make_lda(gd, german_features, 'german_lda', 'german LDA')

In [ ]:
#start = time.time()
#X_LDA = LDA(n_components=3).fit_transform(x, german_data['libretto_or_genre'])
#print('Duration: {} seconds'.format(time.time() - start))


In [ ]:
make_lda(german_data_1770_1819, german_features, "german_lda_1770_1819", "german_lda_1770_1819")

In [ ]:
make_lda(german_data_1820_1869, german_features, "german_lda_1820_1869", "german_lda_1820_1869")

In [ ]:
make_lda(german_data_1870_1920, german_features, "german_lda_1870_1920","german_lda_1870_1920")

### French LDA

In [ ]:
make_lda(filtered_french_data, french_features, 'french_lda', 'French LDA')

In [ ]:
make_lda(french_data_1620_1669, french_features, "french_lda_1620_1669", "french_lda_1620_1669")

In [ ]:
make_lda(french_data_1670_1719, french_features, "french_lda_1670_1719", "french_lda_1670_1719")

In [ ]:
make_lda(french_data_1720_1769, french_features, "french_lda_1720_1769", "french_lda_1720_1769")

In [ ]:
make_lda(french_data_1770_1819, french_features, "french_lda_1770_1819", "french_lda_1770_1819")

In [ ]:
make_lda(french_data_1820_1889, french_features, "french_lda_1820_1889", "french_lda_1820_1889")

### LDA feature importance and interpretation

1. LDA with feature vectors plotted:

In [ ]:
def plot_2d_vectors(df, features, component1, component2, scalings, filename, title):
    feature_vectors = []
    for i in range(len(features)):
        new_feature_vector = go.Scatter(x = [0,scalings[i][0]],
                     y = [0,scalings[i][1]],
                     marker = dict(size = 1,
                                    color = "black"),
                     line = dict(color = "violet",
                                width = 6),
                     name = features[i],
                     text = features[i],
                     mode = 'lines+markers+text'
                     )
        feature_vectors.append(new_feature_vector)
    

    full_plot = go.Scatter(
        x = component1,
        y = component2,
        mode='markers',
        marker=dict(
            size=20,
            color=df['color'], #set color equal to a variable
            colorscale='Rainbow', # one of plotly colorscales
            showscale=True,
            line_width=1
        ),
        text=df['name']
    )
    all_data_to_plot = [full_plot]
    all_data_to_plot.extend(feature_vectors)
    fig = go.Figure(data=all_data_to_plot)
    fig.update_layout(margin=dict( l=100,r=100,b=100,t=100),width=2000,height=1200, font=dict(size=18),
                      title=title)                 
    fig.layout.template = 'plotly'
        
    fig.write_image("output-images/" + filename + ".png",scale=2)
    fig.show()

In [ ]:
def make_lda(df, features, filename='some LDA', title='some LDA'):
    x = df.loc[:, features].values
    x = StandardScaler().fit_transform(x)
    """LDA is supervised; 
    it tries to differentiate classes; 
    so we need to give it classes; in our case they are libretto_or_genre: 
    here two types of libretti are not differentiated. 
    But we color them differently in the viz"""
    X_LDA_fitted = LDA(n_components=2).fit(x, df['libretto_or_genre'])
    X_LDA = X_LDA_fitted.transform(x)
    scalings = X_LDA_fitted.scalings_
    plot_2d_vectors(df, features, X_LDA[:, 0], X_LDA[:, 1], scalings, filename, title)


In [ ]:
make_lda(gd, german_features, 'german_lda_features', 'German LDA with feature vectors')

In [ ]:
#start = time.time()
#X_LDA = LDA(n_components=3).fit_transform(x, german_data['libretto_or_genre'])
#print('Duration: {} seconds'.format(time.time() - start))


In [ ]:
make_lda(german_data_1770_1819, german_features, "german_lda_1770_1819_w_features", "german_lda_1770_1819 with features")

In [ ]:
make_lda(german_data_1820_1869, german_features, "german_lda_1820_1869_w_features", "german_lda_1820_1869_w_features")

In [ ]:
make_lda(german_data_1870_1920, german_features, "german_lda_1870_1920_w_features","german_lda_1870_1920_w_features")

### French LDA

In [ ]:
make_lda(filtered_french_data, french_features, 'french_lda_with_features', 'French LDA with feature vectors')

In [ ]:
make_lda(french_data_1620_1669, french_features, "french_lda_1620_1669_w_features", "french_lda_1620_1669_w_features")

In [ ]:
make_lda(french_data_1670_1719, french_features, "french_lda_1670_1719_w_features", "french_lda_1670_1719_w_features")

In [ ]:
make_lda(french_data_1720_1769, french_features, "french_lda_1720_1769_w_features", "french_lda_1720_1769_w_features")

In [ ]:
make_lda(french_data_1770_1819, french_features, "french_lda_1770_1819_w_features", "french_lda_1770_1819_w_features")

In [ ]:
make_lda(french_data_1820_1889, french_features, "french_lda_1820_1889_w_features", "french_lda_1820_1889_w_features")

### Median values for significant features

German

In [ ]:
gdm = filtered_german_data.groupby('libretto_or_genre').median()
gdm[
    ['word_count_stage', 
    'word_count_sp',
    'num_of_segments', 
    'num_of_person_groups',
    'average_degree',
     'density']
    ]

French

In [ ]:
fdm = filtered_french_data.groupby('libretto_or_genre').median()
fdm[
    ['word_count_stage', 
    'word_count_sp',
    'num_of_speakers', 
    'num_connected_components',
    'average_degree',
     'average_clustering',
     'density']
    ]

In [ ]:
filtered_french_data['num_connected_components'].hist(bins=10)

In [ ]:
fdm = filtered_french_data.groupby('libretto_or_genre').mean()
fdm[
    ['word_count_stage', 
    'word_count_sp',
    'num_of_speakers', 
    'num_connected_components',
    'average_degree',
     'average_clustering',
     'density']
    ]

### Compute distance between points
... to see if libretti are getting nearer and nearer through centuries (genrification process).

In [ ]:
# the coordinates we need are inside the make_lda_function
# and called as X_LDA[:, 0] and X_LDA[:, 1]
# first we store them in some way

def get_libretti_coord_from_lda(df, features):


    df = df.reset_index()

    x = df.loc[:, features].values
    x = StandardScaler().fit_transform(x)
    
    X_LDA_fitted = LDA(n_components=2).fit(x, df['libretto_or_genre']) #something like this

    X_LDA = X_LDA_fitted.transform(x)
    #scalings = X_LDA_fitted.scalings_


    #here we should get only data on libretti


    #we should store all pairs of x, y in arrays (one for each libretto) 
    libretti_index = df[df['is_real_libretto'] == True].index

    #list of libretti 
    #print(len(X_LDA))
    #print(len(libretti_index))
    libretti_coords_list = [X_LDA[i] for i in libretti_index]
    #print(len(libretti_coords_list))
    return libretti_coords_list

In [ ]:
from itertools import combinations

In [ ]:
#compute distance for each pair of libretti
#see https://datascienceparichay.com/article/distance-between-two-points-python/

def get_mean_dist_from_coord_list(coord_list):
    distances_list = []
    dot_pairs = combinations(coord_list, 2) # all pairs of dots in the list
    for pair in dot_pairs:
        distance = np.linalg.norm(pair[0]-pair[1])
        distances_list.append(distance)

    #mean of all distances
    #the lower it is, the nearer the libretti should be

    return np.mean(distances_list) 


In [ ]:
def lib_mean_dist(df, title):
    if 'german' in title:
        features = german_features
    else:
        features = french_features
    coord_list = get_libretti_coord_from_lda(df, features)
    mean_dist = get_mean_dist_from_coord_list(coord_list)
    print(title)
    print('mean LDA distance between libretti:', mean_dist)

In [ ]:
titles_and_dfs = {'german 1770 - 1819':german_data_1770_1819,
'german 1820 - 1869':german_data_1820_1869,
'german 1870 - 1920':german_data_1870_1920,
'french 1620 - 1669':french_data_1620_1669,
'french 1670 - 1719':french_data_1670_1719,
'french 1720 - 1769':french_data_1720_1769,
'french 1770 - 1819':french_data_1770_1819,
'french 1820 - 1889':french_data_1820_1889}

In [ ]:
for title in titles_and_dfs:
    lib_mean_dist(titles_and_dfs[title], title)


### Distances between periods on one single LDA model

In [ ]:
titles_and_dates = [
    {'german 1770 - 1819':(1770,1819),
     'german 1820 - 1869':(1820,1869),
     'german 1870 - 1920':(1870,1920)
    },
    {'french 1620 - 1669':(1620,1669),
      'french 1670 - 1719':(1670,1719),
      'french 1720 - 1769':(1720,1769),
      'french 1770 - 1819':(1770,1819),
      'french 1820 - 1889':(1820,1889)
    }
]

In [ ]:
def output_X_LDA(df, features):
    x = df.loc[:, features].values
    x = StandardScaler().fit_transform(x)
    
    X_LDA_fitted = LDA(n_components=2).fit(x, df['libretto_or_genre'])

    X_LDA = X_LDA_fitted.transform(x)
    return X_LDA
    #scalings = X_LDA_fitted.scalings_

In [ ]:
def get_libretti_for_timeframe_coord_from_lda(df, XLDA, start, end):


    #here we should get only data on libretti


    #we should store all pairs of x, y in arrays (one for each libretto) 
    libretti_index = df[(df['is_real_libretto'] == True) & 
                        (df['year_normalized'] >= start) & 
                        (df['year_normalized'] <= end)].index


    print('Number of libretti:', len(libretti_index))
    libretti_coords_list = [XLDA[i] for i in libretti_index]
    #print(len(libretti_coords_list))
    return libretti_coords_list

In [ ]:
def lib_mean_dist(df, title, timeframe, XLDA):
    coord_list = get_libretti_for_timeframe_coord_from_lda(df, 
                                                           XLDA, 
                                                           timeframe[0], 
                                                           timeframe[1])
    mean_dist = get_mean_dist_from_coord_list(coord_list)
    print('mean LDA distance between libretti:', mean_dist)
    print()

In [ ]:
dfs = [gd, fd]
feats = [german_features, french_features]
for index, df in enumerate(dfs):
    df = df.reset_index()
    XLDA = output_X_LDA(df, feats[index])
    current_titles_and_dates = titles_and_dates[index]
    for title in current_titles_and_dates:
        print(title)
        lib_mean_dist(df, title, current_titles_and_dates[title], XLDA)


trying median

In [ ]:
def lib_median_dist(df, title, timeframe, XLDA):
    coord_list = get_libretti_for_timeframe_coord_from_lda(df, 
                                                           XLDA, 
                                                           timeframe[0], 
                                                           timeframe[1])
    median_dist = get_median_dist_from_coord_list(coord_list)
    print('median LDA distance between libretti:', median_dist)
    print()

In [ ]:
#compute distance for each pair of libretti
#see https://datascienceparichay.com/article/distance-between-two-points-python/

def get_median_dist_from_coord_list(coord_list):
    distances_list = []
    dot_pairs = combinations(coord_list, 2) # all pairs of dots in the list
    for pair in dot_pairs:
        distance = np.linalg.norm(pair[0]-pair[1])
        distances_list.append(distance)

    #mean of all distances
    #the lower it is, the nearer the libretti should be

    return np.median(distances_list) 


In [ ]:
dfs = [gd, fd]
feats = [german_features, french_features]
for index, df in enumerate(dfs):
    df = df.reset_index()
    XLDA = output_X_LDA(df, feats[index])
    current_titles_and_dates = titles_and_dates[index]
    for title in current_titles_and_dates:
        print(title)
        lib_median_dist(df, title, current_titles_and_dates[title], XLDA)

### Distances on original features

In [ ]:
from scipy import spatial


In [ ]:
#from sklearn import metrics

In [ ]:
#for df in dfs:
#index = filtered_german_data['name']
def get_mean_euclidean(df, features):
    x = df.loc[:, features].values
    x = StandardScaler().fit_transform(x)
    #distances = metrics.pairwise.euclidean_distances(x)
    distances = spatial.distance.pdist(x)

    print(np.mean(distances))

In [ ]:
def get_mean_euclidean_time_slice(df, scalings, features, timeframe):
    #distances = metrics.pairwise.euclidean_distances(x)
    start = timeframe[0]
    end = timeframe[1]
    period_libretti_index = df[(df['is_real_libretto'] == True) & 
                        (df['year_normalized'] >= start) & 
                        (df['year_normalized'] <= end)].index
    period_libretti_vectors_list = [scalings[i] for i in period_libretti_index]
    print('number of libretti:', len(period_libretti_vectors_list))
    distances = spatial.distance.pdist(period_libretti_vectors_list)
    print('mean euclidean distance between normalized features:', np.mean(distances))
    print()

In [ ]:
for index, df in enumerate(dfs):
    current_titles_and_dates = titles_and_dates[index]
    features = feats[index]
    x = df.loc[:, features].values
    x = StandardScaler().fit_transform(x)
    df = df.reset_index()
    for title in current_titles_and_dates:
        print(title)
        timeframe = current_titles_and_dates[title]
        get_mean_euclidean_time_slice(df, x, features, timeframe)


## Boxplots for individual features

#### 1. number of segments in libretti (attr+mark) vs. the rest
hypothesis: libretti should have less segments because they use act as the main unit

In [ ]:
#code

#### German

In [ ]:
german_features

In [ ]:
gd.boxplot(column='num_of_segments', 
                             by='is_real_libretto',
                             figsize = (15, 15))

German without ourliers:

In [ ]:
german_no_num_seg_outliers = gd[(np.abs(stats.zscore(gd['num_of_segments'])) < 3)]

In [ ]:
german_no_num_seg_outliers.boxplot(column='num_of_segments', 
                             by='is_real_libretto',
                             figsize = (15, 15))

In [ ]:
#german_no_num_seg_outliers['libretto_or_genre'] = german_no_num_seg_outliers['libretto_or_genre'].replace('Libretto (attributed)', 
                                                                                                          #'Libretto')

In [ ]:
german_no_num_seg_outliers.boxplot(column='num_of_segments', 
                             by='libretto_or_genre',
                             figsize = (15, 15))

#### Word Count stage

In [ ]:
gd.boxplot(column='word_count_stage', 
                             by='libretto_or_genre',
                             figsize = (15, 15))

In [ ]:
german_no_wcstage_outliers = gd[(np.abs(stats.zscore(gd['word_count_stage'])) < 3)]

In [ ]:
german_no_wcstage_outliers.boxplot(column='word_count_stage', 
                             by='libretto_or_genre',
                             figsize = (15, 15))

In [ ]:
filtered_german_data.groupby('libretto_or_genre').median()['word_count_stage'].plot.bar()

In [ ]:
german_no_num_seg_outliers.columns

In [ ]:
german_no_num_seg_outliers.boxplot(column='word_count_sp', 
                             by='genre_with_libretto_subgenres',
                             figsize = (15, 15))

#### French

In [ ]:
french_features

In [ ]:
fd.boxplot(column='num_of_segments', 
                             by='is_real_libretto',
                             figsize = (15, 15))

In [ ]:
french_no_num_seg_outliers = fd[(np.abs(stats.zscore(fd['num_of_segments'])) < 3)]

In [ ]:
french_no_num_seg_outliers.boxplot(column='num_of_segments', 
                             by='is_real_libretto',
                             figsize = (15, 15))

#### split by genre

In [ ]:
french_no_num_seg_outliers.boxplot(column='num_of_segments', 
                             by='libretto_or_genre',
                             figsize = (15, 15))

Word count stage

In [ ]:
filtered_french_data.boxplot(column='word_count_stage', 
                             by='libretto_or_genre',
                             figsize = (15, 15))

In [ ]:
french_no_wcstage_outliers = fd[(np.abs(stats.zscore(fd['word_count_stage'])) < 3)]

In [ ]:
french_no_wcstage_outliers.boxplot(column='word_count_stage', 
                             by='libretto_or_genre',
                             figsize = (15, 15))

In [ ]:
fd.groupby('libretto_or_genre').median()['word_count_stage'].plot.bar()

#### 2. NORMALISED word_count_sp in libretti (attr+mark) vs. the rest

hypothesis: libretti should be shorter

In [ ]:
#code

In [ ]:
filtered_german_data[['name','word_count_text',
                     'word_count_sp',
                     'word_count_stage']]

Gotta decide which features to take

In [ ]:
# code

#### 3. DRAMA CHANGE RATE in libretti (attr+mark) vs. the rest

* a rough proxy for dramatic complexity 
* as implemented here https://dh2017.adho.org/abstracts/071/071.pdf
* is it possible to grab it via API?



In [ ]:
#code

## Statistical Testing

### German

### sp

In [ ]:
nonlibsp = gd[gd['is_real_libretto']==False]['word_count_sp']

In [ ]:
libsp = gd[gd['is_real_libretto']==True]['word_count_sp']

In [ ]:
## checking that the distributions are not normal
w, pvalue = stats.shapiro(nonlibsp)
w, pvalue

In [ ]:
if pvalue < 0.05:
    print('All good, not normal as we expected ✅')
else:
    print('WARNING NORMAL DIST YOU NEED DIFFERENT TEST')

In [ ]:
## checking that the distributions are not normal
w, pvalue = stats.shapiro(libsp)
w, pvalue

In [ ]:
if pvalue < 0.05:
    print('All good, not normal as we expected ✅')
else:
    print('WARNING NORMAL DIST YOU NEED DIFFERENT TEST')

In [ ]:
result = stats.mannwhitneyu(x=nonlibsp, y=libsp, alternative = 'two-sided')

In [ ]:
result

In [ ]:
if result.pvalue < 0.05:
    print('Difference is significant! ✅')
else:
    print('Not significant❌')

Wrap this into a function

In [ ]:
def test_significance(data, feature_name, binary_split_criteria):
    A = data[data[binary_split_criteria]==False][feature_name]
    B = data[data[binary_split_criteria]==True][feature_name]
    norm_counter = 0
    for sample in (A,B):
        w, pvalue = stats.shapiro(sample)
        if pvalue > 0.05:
            norm_counter+=1

    if norm_counter == 2:
        #print('Two normal distrs, using unpaired T-test')
        result = stats.ttest_ind(A, B)
    elif norm_counter == 1:
        #print('One distr is normal and one is not!')
        #print('Using Wilcoxon rank sum test')
        result = stats.mannwhitneyu(x=A, y=B, alternative = 'two-sided')
    else:
        #print('Both non-Normal, using Wilcoxon rank sum test')
        result = stats.mannwhitneyu(x=A, y=B, alternative = 'two-sided')
    
    #if result.pvalue < 0.05:
    #    print('Difference is significant! ✅ pvalue is', result.pvalue)
        
    #else:
    #    print('Not significant❌, pvalue is', result.pvalue)
    
    print(f"{feature_name} & {round(result.pvalue, 20)} & some number \\\\") #format(result.pvalue, '.8f')
    print(format(result.pvalue, '.10f'))


In [ ]:
for feature in list_features_pyd: #list_features_pyd: #
    #print('Testing', feature)
    test_significance(gd, feature, 'is_real_libretto')
    #print()

### French

In [ ]:
fd = filtered_french_data

In [ ]:
for feature in list_features_pyd:
    #print('Testing', feature)
    test_significance(fd, feature, 'is_real_libretto')
    #print()

##  Important features dynamic plots (by timeframes)

In [ ]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [ ]:
def median_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.median(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [ ]:
#ci1 = mean_confidence_interval(intra_distances)
#plot_ci(ci1[0], ci1[1], ci1[2], height.mean(), pyplot)

In [ ]:
def assign_period(year, lang='de'):
    if 1600 <= year <= 1669:
        return (1600,1669)
    elif 1670 <= year <= 1719:
        return (1670,1719)
    elif 1720 <= year <= 1769:
        return (1720,1769)
    elif 1770 <= year <= 1819:
        return (1770,1819)        
    elif 1820 <= year <= 1869:
        return (1820,1869)
    else: 
        if lang=='de': # latest period exists for german only
            return (1870,1920)
        else: # but for french we attach them to a bigger 1820-1889 timeframe
            return (1820,1869)

#### German

In [ ]:
gd_lt = gd[(gd['year_normalized']>1769) & (gd['libretto_or_genre'] != 'Tragicomedy')]

In [ ]:

gd_lt['period'] = gd_lt['year_normalized'].apply(assign_period)

In [ ]:
#gd[['period', 'year_normalized']]

In [ ]:
from matplotlib import colors as mplcolors

In [ ]:
colors_for_plot = ['blue', 'aquamarine', 'orange', 'red']

In [ ]:
colors = mplcolors.ListedColormap(colors_for_plot, name='libretti_colors', N=None)

In [ ]:
counts = gd_lt.groupby(['period', 'genre_with_libretto_subgenres']).count().unstack(level=-1)['word_count_sp']

In [ ]:
counts.columns

In [ ]:
toplot = gd_lt.groupby(['period', 'genre_with_libretto_subgenres']).median().unstack(level=-1)['word_count_sp']

In [ ]:
toplot.index

In [ ]:
toplot = gd_lt.groupby(['period', 'genre_with_libretto_subgenres']).median().unstack(level=-1)['word_count_sp']

toplot.plot(figsize=(10, 18), colormap = colors)
ax = plt.gca()
#print(ax.get_xticks())
#for line in ax.get_lines():
#    print(line.get_data())

counts = gd_lt.groupby(['period', 'genre_with_libretto_subgenres']).count().unstack(level=-1)['word_count_sp']
for column in toplot.columns:
    #print(column)
    ys = toplot[column]
    xs = toplot.index
    values = counts[column]
    for index, x in enumerate(xs):
        #print(x)
        #print(ys[index])
        #print(ys[index])
        #print(str(values[index]))
        plt.text(index, ys[index], f'no. plays: {str(values[index])}', fontsize=10) #str(ys[index])


ax.set_ylim(ymin=0)


In [ ]:
import random

In [ ]:
import scipy

In [ ]:
def get_lineplots(data, feature_name, shift, comedyup=False, tragedyup=False, get_mean=False):
    
    
    toplot = data.groupby(['period', 'genre_with_libretto_subgenres']).median().unstack(level=-1)[feature_name]
    if get_mean:
        toplot = data.groupby(['period', 'genre_with_libretto_subgenres']).mean().unstack(level=-1)[feature_name]

    #plt.plot(label='line with marker')
    toplot.plot(marker="o", figsize=(10, 15), colormap = colors)
    
    ax = plt.gca()
    #print(ax.get_xticks())
    #for line in ax.get_lines():
    #    print(line.get_data())
    colors_for_num = ['blue', 'black', 'black', 'red']#'orange', 'red']
    

    counts = data.groupby(['period', 'genre_with_libretto_subgenres']).count().unstack(level=-1)[feature_name]
    for cindex, column in enumerate(toplot.columns):
        #print(column)
        ys = toplot[column]
        #print('numbers for comedy', ys)
        xs = toplot.index
        values = counts[column]
        realshift = shift
        std = ys.std()
        mean = ys.mean()
        p025 = ys.quantile(0.025)
        p975 = ys.quantile(0.975)
        errorexes = list(range(len(xs)))

        interval_ups = []
        interval_downs = []
        for period in xs:

        #current_confint = [ci[1], ci[2]]
            if get_mean:
                current_confint = mean_confidence_interval(data[(data['genre_with_libretto_subgenres'] == column) &
                                                                (data['period'] == period)][feature_name]
                                                                )
                
                
            else:
                current_confint = median_confidence_interval(data[(data['genre_with_libretto_subgenres'] == column) &
                                                                (data['period'] == period)][feature_name]
                                                                )
            interval_downs.append(current_confint[1])
            interval_ups.append(current_confint[2])
          
        #print(column, interval_ups, interval_downs)
        errorexes = [x+((cindex-2)*0.02) for x in errorexes]
        #errorys = y. 
        dummyups = [2,2,2]
        dummydowns = [1,1,1]
        #print(list(ys))

        print('real values', list(ys))
        print('upper boundaries', interval_ups)
        ups = np.array(interval_ups) - np.array(ys)
        print('ups', ups)
        print()
        print('lower boundaries',interval_downs)
        downs =  np.array(ys) - np.array(interval_downs) 
        print('downs', downs)
        plt.errorbar(errorexes, ys, xerr=0, yerr=[downs, ups], linestyle='', 
                     color = colors_for_plot[cindex]) # label = 'errorbar'

        if comedyup and 'Comedy' in column:
            realshift = -shift
        if tragedyup and 'Tragedy' in column:
            realshift = -shift
        for index, x in enumerate(xs):
            #print(x)
            #print(ys[index])
            #print(ys[index])
            #print(str(values[index]))
            #shift = 400
            #if values[index] == 32:
            #    shift=-200
            
            plt.text(index, ys[index]-realshift, str(values[index]), fontsize=10, color=colors_for_num[cindex]) #str(ys[index])


    ax.set_ylim(ymin=0)
    
    plt.savefig(f'{feature_name}_with_conf_int.jpeg', edgecolor='black', dpi=400)


In [ ]:
get_lineplots(gd_lt, 'average_degree', 0.2, tragedyup=True)

In [ ]:
get_lineplots(gd_lt, 'word_count_sp', 0.2, tragedyup=True)

In [ ]:
gd_lt.columns

In [ ]:
get_lineplots(gd_lt, 'num_of_person_groups', 0.2, tragedyup=True) 

In [ ]:
get_lineplots(gd_lt, 'word_count_sp', 300, comedyup=True)

In [ ]:
gd_lt.groupby(['period', 'genre_with_libretto_subgenres']).median().unstack(level=-1)['num_of_speakers'].plot(figsize=(10, 8), 
                                                                                                            colormap = colors)
ax = plt.gca()
ax.set_ylim(ymin=0)


In [ ]:
get_lineplots(gd_lt, 'num_of_speakers', 0.4)

In [ ]:
get_lineplots(gd_lt, 'num_of_person_groups', 0.4)

In [ ]:
gd_lt.groupby(['period', 'genre_with_libretto_subgenres']).mean().unstack(level=-1)['num_of_person_groups'].plot(figsize=(10, 8), 
                                                                                                            colormap = colors)
ax = plt.gca()
ax.set_ylim(ymin=0)


#### French

In [ ]:
fd['period'] = fd['year_normalized'].apply(assign_period, lang='fr')

In [ ]:
fd_lt = fd[(fd['libretto_or_genre'] != 'Tragicomedy')]

In [ ]:
fd_lt.groupby(['period', 'genre_with_libretto_subgenres']).median().unstack(level=-1)['word_count_stage'].plot(figsize=(10, 8), 
                                                                                                            colormap = colors)
ax = plt.gca()
ax.set_ylim(ymin=0)

In [ ]:
get_lineplots(fd_lt, 'word_count_stage', 12, comedyup=True)

In [ ]:
french_data_1670_1719[french_data_1670_1719['genre_with_libretto_subgenres']=='Non-comic libretto'][['first_author','title', 'num_of_speakers']]

In [ ]:
fd_lt.groupby(['period', 'genre_with_libretto_subgenres']).median().unstack(level=-1)['num_of_speakers'].plot(figsize=(10, 8), 
                                                                                                            colormap = colors
                                                                                                              )
ax = plt.gca()
ax.set_ylim(ymin=0)

In [ ]:
get_lineplots(fd_lt, 'num_of_speakers', 0.35, comedyup=True)

In [ ]:
fd_lt.groupby(['period', 'genre_with_libretto_subgenres']).median().unstack(level=-1)['word_count_sp'].plot(figsize=(10, 8), 
                                                                                                            colormap = colors
                                                                                                              )
ax = plt.gca()
ax.set_ylim(ymin=0)

In [ ]:
get_lineplots(fd_lt, 'word_count_sp', 400, comedyup=True)

In [ ]:
fd_lt.groupby(['period', 'genre_with_libretto_subgenres']).median().unstack(level=-1)['density'].plot(figsize=(10, 8), 
                                                                                                            colormap = colors
                                                                                                              )
ax = plt.gca()
ax.set_ylim(ymin=0)

In [ ]:
get_lineplots(fd_lt, 'density', 0.02, comedyup=True, tragedyup=True)

In [ ]:
fd_lt.groupby(['period', 'genre_with_libretto_subgenres']).mean().unstack(level=-1)['num_connected_components'].plot(figsize=(10, 8), 
                                                                                                            colormap = colors
                                                                                                              )
ax = plt.gca()
ax.set_ylim(ymin=0)

In [ ]:
get_lineplots(fd_lt, 'num_connected_components', 0.05, tragedyup=True, get_mean=True)#, comedyup=True, tragedyup=True)

In [ ]:
french_features

In [ ]:
fd_lt.groupby(['period', 'genre_with_libretto_subgenres']).median().unstack(level=-1)['diameter'].plot(figsize=(10, 8), 
                                                                                                            colormap = colors
                                                                                                              )
ax = plt.gca()
ax.set_ylim(ymin=0)

In [ ]:
get_lineplots(fd_lt, 'diameter', 0.15, comedyup=True, tragedyup=True)

In [ ]:
fd_lt.groupby(['period', 'genre_with_libretto_subgenres']).median().unstack(level=-1)['num_of_segments'].plot(figsize=(10, 8), 
                                                                                                            colormap = colors
                                                                                                              )
ax = plt.gca()
ax.set_ylim(ymin=0)

In [ ]:
french_data_1720_1769[french_data_1720_1769['genre_with_libretto_subgenres'] == 'Comic libretto'][['first_author', 'title', 'num_of_segments']]

In [ ]:
fd_lt.groupby(['period', 'genre_with_libretto_subgenres']).median().unstack(level=-1)['max_degree'].plot(figsize=(10, 8), 
                                                                                                            colormap = colors
                                                                                                              )
ax = plt.gca()
ax.set_ylim(ymin=0)

In [ ]:
fd_lt.groupby(['period', 'genre_with_libretto_subgenres']).median().unstack(level=-1)['average_path_length'].plot(figsize=(10, 8), 
                                                                                                            colormap = colors
                                                                                                              )
ax = plt.gca()
ax.set_ylim(ymin=0)


## Scatterplots

FILTER THE SUBSET BY LIBRETTO PERIOD

In [ ]:
years = fd[fd['is_real_libretto'] == True]['year_normalized']
earliest_french_libr = years.min()
earliest_french_libr

In [ ]:
latest_french_libr = years.max()
latest_french_libr

In [ ]:
fd_sc = fd[(fd['libretto_or_genre'] != 'Tragicomedy') & 
            (fd['year_normalized'] >= earliest_french_libr) &
            (fd['year_normalized'] <= latest_french_libr)]

In [ ]:
fd_sc['year_normalized'].describe()

In [ ]:
gyears = gd[gd['is_real_libretto'] == True]['year_normalized']
earliest_g_libr = gyears.min()
earliest_g_libr

In [ ]:
latest_g_libr = gyears.max()
latest_g_libr

In [ ]:
gd_sc = gd[(gd['libretto_or_genre'] != 'Tragicomedy') & 
            (gd['year_normalized'] >= earliest_g_libr) &
            (gd['year_normalized'] <= latest_g_libr)]

In [ ]:
gd_sc['year_normalized'].describe()

#### Basic scatterplot

In [ ]:
plt.figure(figsize=(10,10))
sns.scatterplot(data=gd_sc, 
                x='year_normalized',
                y='word_count_sp', 
                hue='genre_with_libretto_subgenres')

In [ ]:
colors_for_plot

#### Lowess

In [ ]:
gd_lt_no_out = gd_sc[(np.abs(stats.zscore(gd_lt['word_count_sp'])) < 3)]
sns.lmplot(data=gd_lt_no_out, 
            x='year_normalized',
            y='word_count_sp', 
            hue='genre_with_libretto_subgenres',
            height=12, aspect=1.5, 
           palette=['blue', 'red', 'aquamarine', 'orange'],
           #logistic=True
           lowess=True,
           #alpha=0.3
           )
plt.savefig('scatter.png', dpi=300)
#plt.show()

wrap in a function

In [ ]:
gd_sc[['genre_with_libretto_subgenres', 'color_subgenres']].head(20)

In [ ]:
def get_lowess_plot(df, parameter):
    df_no_out = df[(np.abs(stats.zscore(df[parameter])) < 3)]
    sns.lmplot(data=df_no_out, 
                x='year_normalized',
                y=parameter, 
                hue='genre_with_libretto_subgenres',
                height=12, aspect=1.5, 
            palette=df['color_subgenres'].unique(),
            #logistic=True
            lowess=True,
            alpha = 0.5
            )
    plt.savefig(f'{parameter}_scatter_lowess.png', dpi=300)

#### Produce lowess german

In [ ]:
get_lowess_plot(gd_sc, 'word_count_sp')

In [ ]:
get_lowess_plot(gd_sc, 'num_of_person_groups')

#### Produce lowess French

In [ ]:
get_lowess_plot(fd_sc, 'word_count_sp')

In [ ]:
get_lowess_plot(fd_sc, 'word_count_stage')

In [ ]:
get_lowess_plot(fd_sc, 'density')

In [ ]:
fd_sc['year_normalized'].describe()

In [ ]:
get_lowess_plot(fd_sc, 'num_of_speakers')

### Mor low-level version of lowess (same result, more complicated implementation, BUT can make only dots transparent)

In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess

In [ ]:
def make_lowess(df, column):
    by_year = df.groupby('year_normalized').mean()
    endog = np.array(by_year[column])
    #print(endog)
    exog = np.array(by_year.index)
    #print(exog)

    smooth = lowess(endog, exog)
    index, data = np.transpose(smooth)

    return pd.Series(data, index=index) 

In [ ]:
def make_scatter_lowess_transp_dots(df, parameter):
    fig = plt.gcf()

    df_no_out = df[(np.abs(stats.zscore(df[parameter])) < 3)]

    scatter_colors = list(df_no_out['color_subgenres'].unique())
    # Change seaborn plot size
    fig.set_size_inches(15, 10)
    sns.scatterplot(data=df_no_out, 
                x='year_normalized',
                y=parameter, 
                hue='genre_with_libretto_subgenres',
                #height=12, aspect=1.5, 
                palette=scatter_colors, 
                alpha = 0.2)

    for index, genre in enumerate(df_no_out['genre_with_libretto_subgenres'].unique()):
        lwss = make_lowess(df_no_out[df_no_out['genre_with_libretto_subgenres'] == genre], 
                        parameter)
        sns.lineplot(x = lwss.index, y = lwss, color=scatter_colors[index], linewidth = 2)
    
    plt.savefig(f'{parameter}_scatter_with_lowess.png')

#### german transparent scatterplots

In [ ]:
make_scatter_lowess_transp_dots(gd_sc, 'word_count_sp')

In [ ]:
make_scatter_lowess_transp_dots(gd_sc, 'num_of_person_groups')

#### french

In [ ]:
make_scatter_lowess_transp_dots(fd_sc, 'num_of_speakers')

In [ ]:
make_scatter_lowess_transp_dots(fd_sc, 'density')

In [ ]:
make_scatter_lowess_transp_dots(fd_sc, 'word_count_stage')

In [ ]:
make_scatter_lowess_transp_dots(fd_sc, 'word_count_sp')

Unused plotly variant

In [ ]:
px.scatter(gd, x='density', y='average_path_length', labels='title')

##  PCA loadings

In [ ]:
def make_pca_with_loadings(df, feature_list, output_filename, title):
    standardized_data = standardize(df, feature_list)
    start = time.time()
    pca = PCA(n_components=2)

    principalComponents = pca.fit_transform(standardized_data)
    print('Duration: {} seconds'.format(time.time() - start))
    principal = pd.DataFrame(data = principalComponents
                , columns = ['principal component 1', 'principal component 2']) #,'principal component 3'])
    plot_2d(df, principalComponents[:, 0],
            principalComponents[:, 1], 
            output_filename, 
            title)
    loadings = pca.components_
    num_pc = pca.n_features_
    pc_list = ["PC"+str(i) for i in list(range(1, num_pc+1))]
    loadings_df = pd.DataFrame.from_dict(dict(zip(pc_list, loadings)))
    loadings_df['variable'] = feature_list
    loadings_df = loadings_df.set_index('variable')
    print(pca.explained_variance_ratio_)
    return loadings_df

In [ ]:
loadings = make_pca_with_loadings(french_data_1670_1719, french_features, 'pcafrench_data_1670_1719', 'pcafrench_data_1670_1719')

In [ ]:
loadings.T.plot.bar(figsize=(15,15))

In [ ]:
french_data_1670_1719[['first_author', 'title', 'num_connected_components']].sort_values('num_connected_components', ascending=False).head(25)

In [ ]:
french_data_1670_1719[french_data_1670_1719['num_connected_components'] == 4][['first_author', 'title', 'subtitle', 'num_connected_components']]

##  TODO: Classifier, checking accuracy and getting feature importance

Expecting this to align well with the statistical test above

In [ ]:
rstate = 42

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
from typing import Tuple

In [ ]:
import copy as cp

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
def analyze_model_classwise(trained_model, features_val, target_val):
    """функция для оценки модели в контексте дисбаланса классов
    рисует матрицу ошибок и считает accuracy для каждого класса
    """
    
    with open('new.csv', 'a', encoding='utf-8') as allfile:
        allfile.write(str(trained_model))
        allfile.write('\n')
        predicted_val = trained_model.predict(features_val)
        print(f'accuracy : {accuracy_score(target_val, predicted_val)}')
        allfile.write(f'accuracy \t{"{:.3f}".format(accuracy_score(target_val, predicted_val))} \n')
        
        
        labels = trained_model.classes_
        cm = confusion_matrix(target_val, predicted_val, labels=labels)


        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=(labels))
        disp.plot(xticks_rotation = 90)
        class_recalls = list(zip(labels, cm.diagonal()/cm.sum(axis=1)))
        class_precisions = list(zip(labels, cm.diagonal()/cm.sum(axis=0)))
        for i, some_class in enumerate(class_recalls):
            this_class = some_class[0]
            p = class_precisions[i][1]
            r = some_class[1]
            f1 = (2*p*r)/(p+r)
            print(f'precision for class {this_class}: {p}')
            print(f'recall for class {this_class}: {r}')
            print(f'f1 for class {this_class}: {f1}')
            allfile.write(f'precision for class {this_class}\t {"{:.3f}".format(p)} \n')
            allfile.write(f'recall_for_class {this_class}\t{"{:.3f}".format(r)} \n')
            allfile.write(f'f1 for class {this_class}\t{"{:.3f}".format(f1)} \n')
        
    #print('ROC AUC')    
    #draw_roc(target_val, predicted_val)

    plt.show()
    

In [ ]:
def cross_val_predict(model, kfold : KFold, X : np.array, y : np.array) -> Tuple[np.array, np.array, np.array]:

    model_ = cp.deepcopy(model)
    
    no_classes = len(np.unique(y))
    
    actual_classes = np.empty([0], dtype=int)
    predicted_classes = np.empty([0], dtype=int)
    predicted_proba = np.empty([0, no_classes]) 

    for train_ndx, test_ndx in kfold.split(X):

        train_X, train_y, test_X, test_y = X[train_ndx], y[train_ndx], X[test_ndx], y[test_ndx]

        actual_classes = np.append(actual_classes, test_y)

        model_.fit(train_X, train_y)
        predicted_classes = np.append(predicted_classes, model_.predict(test_X))

        try:
            predicted_proba = np.append(predicted_proba, model_.predict_proba(test_X), axis=0)
        except:
            predicted_proba = np.append(predicted_proba, np.zeros((len(test_X), no_classes), dtype=float), axis=0)

    return actual_classes, predicted_classes, predicted_proba

In [ ]:
def plot_confusion_matrix(actual_classes : np.array, predicted_classes : np.array, sorted_labels : list, lang='German'):

    matrix = confusion_matrix(actual_classes, predicted_classes, labels=sorted_labels)
    
    plt.figure(figsize=(12.8,6))
    sns.heatmap(matrix, annot=True, xticklabels=sorted_labels, yticklabels=sorted_labels, cmap="Blues", fmt="g")
    plt.xlabel('Predicted'); plt.ylabel('Actual'); plt.title(f'Combined Confusion Matrix for all folds of the 5-Fold Cross-Validation on {lang} corpus')


    
    plt.savefig(f'confusion_matrix_{lang}_balanced_kfold_combined.png',
                dpi=300)
    plt.show()

    print(f'accuracy : {accuracy_score(actual_classes, predicted_classes)}')

    cm = matrix
    class_recalls = list(zip(sorted_labels, cm.diagonal()/cm.sum(axis=1)))
    class_precisions = list(zip(sorted_labels, cm.diagonal()/cm.sum(axis=0)))
    for i, some_class in enumerate(class_recalls):
        this_class = some_class[0]
        p = class_precisions[i][1]
        r = some_class[1]
        f1 = (2*p*r)/(p+r)
        print(f'precision for class {this_class}: {p}')
        print(f'recall for class {this_class}: {r}')
        print(f'f1 for class {this_class}: {f1}')

In [ ]:
def classify(classifier, df, feature_list, classification_parameter):
    features = standardize(df, feature_list)
    target = df[classification_parameter]
    scores = cross_val_score(classifier, features, target)
    return(scores)

In [ ]:
def assess_best_rf(df, features_list, classification_parameter):
    best = 0 
    for i in range(100,1000,50):
        model = RandomForestClassifier(n_estimators = i, random_state = rstate)
        cross_val_scores = classify(model, df, features_list, classification_parameter)
        meanscore = np.mean(cross_val_scores)
        print(meanscore)
        if meanscore > best:
            best = meanscore
            best_n_est = i
            best_rf_model = model

    print(f'Best result: {best} Produced by model with {best_n_est} estimators')



In [ ]:
def assess_best_brf(df, features_list, classification_parameter):
    best = 0 
    for i in range(100,1000,50):
        model = BalancedRandomForestClassifier(n_estimators = i, random_state = rstate)
        cross_val_scores = classify(model, df, features_list, classification_parameter)
        meanscore = np.mean(cross_val_scores)
        print(meanscore)
        if meanscore > best:
            best = meanscore
            best_n_est = i
            best_rf_model = model

    print(f'Best result: {best} Produced by model with {best_n_est} estimators')



In [ ]:
def assess_best(df, features_list, classification_parameter, balanced=True):
    best = 0 
    for i in range(100,800,50):
        if balanced:
            model = BalancedRandomForestClassifier(n_estimators = i, random_state = rstate)
        else:
            model = RandomForestClassifier(n_estimators = i, random_state = rstate)
        cross_val_scores = classify(model, df, features_list, classification_parameter)
        meanscore = np.mean(cross_val_scores)
        print(meanscore)
        if meanscore > best:
            best = meanscore
            best_n_est = i
            best_rf_model = model
    print(f'Best result: {best} Produced by model with {best_n_est} estimators')
    return best_rf_model

In [ ]:
def get_feature_importances(df, good_model, target_col, features_list, corpusname):
    features = standardize(df, features_list)
    target = df[target_col]
    features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                                    target, 
                                                                                    test_size = 0.3,
                                                                                    random_state=rstate)
    result = permutation_importance(
    good_model, features_test, target_test, n_repeats=10, random_state=rstate, n_jobs=2)
    forest_importances = pd.Series(result.importances_mean, index=features_list)
    fig, ax = plt.subplots()
    forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
    ax.set_title(f"Feature importances using permutation on full model on {corpusname} corpus")
    ax.set_ylabel("Mean accuracy decrease")
    fig.set_size_inches(10, 6)
    fig.tight_layout()
    plt.savefig('latest_features_imp.png', dpi=300)
    plt.show()


In [ ]:
def fully_automated_model_assessment(df, 
                                     features_list, 
                                     classification_parameter, 
                                     balanced=True,
                                     lang='German'):
    model = assess_best(df, features_list, classification_parameter, balanced=balanced)
    df = df.reset_index(drop=True)
    X = standardize(df, features_list)
    y = df[classification_parameter]

    ## cross val conf matrix
    kfold = KFold(n_splits=5, random_state=rstate, shuffle=True)
    class_labels = df[classification_parameter].unique()
    class_labels.sort()
    actual_classes, predicted_classes, _ = cross_val_predict(model, kfold, X, y)
    plot_confusion_matrix(actual_classes, predicted_classes, class_labels, lang=lang)

    ## importances 

    features_train, features_test, target_train, target_test = train_test_split(X, 
                                                                            y, 
                                                                            test_size = 0.3,
                                                                            random_state=rstate)
    
    model.fit(features_train, target_train)
    get_feature_importances(df, 
                            model, 
                            classification_parameter, 
                            features_list, 
                            lang)


### German class-wise with cross val optimization, cross val testing and cross val conf matrix

In [ ]:
gd_lt = gd[gd['libretto_or_genre'] != 'Tragicomedy']

In [ ]:
gd_lt['genre_with_libretto_subgenres'].unique()

In [ ]:
fully_automated_model_assessment(gd_lt,
                                 german_features,
                                 'genre_with_libretto_subgenres')

### German 4 class imbalanced

In [ ]:
fully_automated_model_assessment(gd_lt,
                                 german_features,
                                 'genre_with_libretto_subgenres',
                                 balanced=False)

### French class-wise with cross val optimization, cross val testing and cross val conf matrix

In [ ]:
fd_lt = fd[fd['libretto_or_genre'] != 'Tragicomedy']

In [ ]:
fully_automated_model_assessment(fd_lt,
                                 french_features,
                                 'genre_with_libretto_subgenres',
                                 lang='French')

#### French 4 class imbalanced

In [ ]:
fully_automated_model_assessment(fd_lt,
                                 french_features,
                                 'genre_with_libretto_subgenres',
                                 lang='French',
                                 balanced=False)

### German binary balanced

In [ ]:
fully_automated_model_assessment(gd_lt,
                                 german_features,
                                 'is_real_libretto')

In [ ]:
assess_best_brf(gd_lt, german_features, 'is_real_libretto')

In [ ]:
good_model = BalancedRandomForestClassifier(n_estimators = 150, random_state = rstate)

In [ ]:
X = standardize(gd_lt, german_features)
y = gd_lt['is_real_libretto']

In [ ]:
features = standardize(gd_lt, german_features)
target = gd_lt['is_real_libretto']
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                                    target, 
                                                                                    test_size = 0.3,
                                                                                    random_state=rstate)

In [ ]:
good_model.fit(features, target)

In [ ]:
get_feature_importances(gd_lt, good_model, 'is_real_libretto', german_features, 'German')

In [ ]:
genres = gd_lt['is_real_libretto'].unique()

In [ ]:
genres.sort()

In [ ]:
genres

In [ ]:
actual_classes, predicted_classes, _ = cross_val_predict(good_model, kfold, X, y)
plot_confusion_matrix(actual_classes, predicted_classes, genres, lang= 'German')

### French binary balanced

In [ ]:
fully_automated_model_assessment(fd_lt,
                                 french_features,
                                 'is_real_libretto', 
                                 lang='French')

### French binary imbalanced

In [ ]:
fully_automated_model_assessment(fd_lt,
                                 french_features,
                                 'is_real_libretto',
                                 balanced = False, 
                                 lang='French')

### German Binary imbalanced

In [ ]:
fully_automated_model_assessment(gd_lt,
                                 german_features,
                                 'is_real_libretto',
                                 balanced = False, 
                                 lang='German')

#### Imbalanced german model, confusion matrix k fold

In [ ]:
assess_best_rf(gd_lt, german_features, 'genre_with_libretto_subgenres')

In [ ]:
good_unbalanced_model = RandomForestClassifier(n_estimators = 500, random_state = rstate)

In [ ]:
X = standardize(gd_lt, german_features)
y = gd_lt['genre_with_libretto_subgenres']

In [ ]:
actual_classes, predicted_classes, _ = cross_val_predict(good_unbalanced_model, kfold, X, y)
plot_confusion_matrix(actual_classes, predicted_classes, genres)

In [ ]:
good_unbalanced_model

In [ ]:
features = standardize(gd_lt, german_features)
target = gd_lt['libretto_or_genre']
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                                    target, 
                                                                                    test_size = 0.3,
                                                                                    random_state=rstate)

In [ ]:
good_unbalanced_model.fit(features, target)

In [ ]:
get_feature_importances(gd_lt, good_unbalanced_model, german_features, 'German')

### Older testing

In [ ]:
fd_lt['genre_with_libretto_subgenres'].unique()

In [ ]:
assess_best_brf(fd_lt, french_features, 'genre_with_libretto_subgenres')

In [ ]:
good_model = BalancedRandomForestClassifier(n_estimators = 250, random_state = rstate)

In [ ]:
fd_lt = fd_lt.reset_index(drop=True)

In [ ]:
X = standardize(fd_lt, french_features)
y = fd_lt['genre_with_libretto_subgenres']


In [ ]:
kfold = KFold(n_splits=5, random_state=rstate, shuffle=True)

In [ ]:
actual_classes, predicted_classes, _ = cross_val_predict(good_model, kfold, X, y)
plot_confusion_matrix(actual_classes, predicted_classes, genres, lang= 'French')

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(X, 
                                                                            y, 
                                                                            test_size = 0.3,
                                                                            random_state=rstate)

In [ ]:
good_model.fit(features_train, target_train)

In [ ]:
get_feature_importances(fd_lt, 
                        good_model, 
                        'genre_with_libretto_subgenres', 
                        french_features, 
                        'French')

In [ ]:
genres = sorted(list(gd_lt['genre_with_libretto_subgenres'].unique()))

In [ ]:
genres

In [ ]:
assess_best_brf(gd_lt, german_features, 'genre_with_libretto_subgenres')

In [ ]:
good_model = BalancedRandomForestClassifier(n_estimators = 300, random_state = rstate)

In [ ]:
gd_lt = gd_lt.reset_index(drop=True)

In [ ]:
X = standardize(gd_lt, german_features)
y = gd_lt['genre_with_libretto_subgenres']


In [ ]:
gd_lt['genre_with_libretto_subgenres'].value_counts()

In [ ]:
kfold = KFold(n_splits=5, random_state=rstate, shuffle=True)

In [ ]:
actual_classes, predicted_classes, _ = cross_val_predict(good_model, kfold, X, y)
plot_confusion_matrix(actual_classes, predicted_classes, genres)

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(X, 
                                                                            y, 
                                                                            test_size = 0.3,
                                                                            random_state=rstate)

In [ ]:
good_model.fit(features_train, target_train)

In [ ]:
get_feature_importances(gd_lt, good_model, 'genre_with_libretto_subgenres', german_features, 'German')

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                                    target, 
                                                                                    test_size = 0.3,
                                                                                    random_state=rstate)

In [ ]:
good_model.fit(features_train, target_train)

In [ ]:
analyze_model_classwise(good_model, features_test, target_test)

In [ ]:
get_feature_importances(gd_lt, good_model, german_features, 'German')

Multiclass (Comedy, Tragedy, Tragcomedy or libretto)

In [ ]:
def classify(classifier, df, feature_list):
    features = standardize(df, feature_list)
    target = df['libretto_or_genre']
    scores = cross_val_score(classifier, features, target)
    return(scores)

In [ ]:
rstate=42

In [ ]:
model = RandomForestClassifier(random_state=rstate)

In [ ]:
result = classify(model, gd, list_features_pyd)

In [ ]:
result

In [ ]:
np.mean(result)

In [ ]:
plt.bar(list(range(len(result))), list(result))
ax = plt.gca()

#ax.set_xlim([xmin, xmax])
ax.set_ylim([0, 1])

In [ ]:
best = 0 

for i in range(100,1000,50):
    model = RandomForestClassifier(n_estimators = i, random_state = rstate)
    cross_val_scores = classify(model, gd, list_features_pyd)
    meanscore = np.mean(cross_val_scores)
    print(meanscore)
    if meanscore > best:
        best = meanscore
        best_n_est = i
        best_rf_model = model

print(f'Best result: {best} Produced by model with {best_n_est} estimators')

In [ ]:
good_model = RandomForestClassifier(n_estimators = 200, random_state = rstate)

In [ ]:
features = standardize(gd, list_features_pyd)
target = gd['libretto_or_genre']
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                                    target, 
                                                                                    test_size = 0.3,
                                                                                    random_state=rstate)

In [ ]:
target_test.value_counts()

In [ ]:
good_model.fit(features_train, target_train)

In [ ]:
analyze_model_classwise(good_model, features_test, target_test)

In [ ]:


result = permutation_importance(
    good_model, features_test, target_test, n_repeats=10, random_state=rstate, n_jobs=2
)

In [ ]:
forest_importances = pd.Series(result.importances_mean, index=list_features_pyd)

In [ ]:
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
fig.set_size_inches(10, 6)
fig.tight_layout()
plt.show()

### Only german features

In [ ]:
best = 0 

for i in range(100,1000,50):
    model = RandomForestClassifier(n_estimators = i, random_state = rstate)
    cross_val_scores = classify(model, gd, german_features)
    meanscore = np.mean(cross_val_scores)
    print(meanscore)
    if meanscore > best:
        best = meanscore
        best_n_est = i
        best_rf_model = model

print(f'Best result: {best} Produced by model with {best_n_est} estimators')

In [ ]:
good_model = RandomForestClassifier(n_estimators = 350, random_state = rstate)

In [ ]:
features = standardize(gd, german_features)
target = gd['libretto_or_genre']
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                                    target, 
                                                                                    test_size = 0.3,
                                                                                    random_state=rstate)

In [ ]:
good_model.fit(features_train, target_train)

In [ ]:
analyze_model_classwise(good_model, features_test, target_test)

In [ ]:
result = permutation_importance(
    good_model, features_test, target_test, n_repeats=10, random_state=rstate, n_jobs=2
)

In [ ]:
forest_importances = pd.Series(result.importances_mean, index=german_features)

In [ ]:
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
fig.set_size_inches(10, 6)
fig.tight_layout()
plt.show()

In [ ]:
gd.boxplot('num_of_person_groups', by='is_real_libretto')

In [ ]:
gd[gd['is_real_libretto'] == True].sort_values('num_of_person_groups', ascending=False)[['first_author', 'title', 'num_of_person_groups', 'year_normalized']].head(25)

Binary

In [ ]:
good_model = RandomForestClassifier(n_estimators = 110, random_state = rstate)

In [ ]:
features = standardize(gd, list_features_pyd)
target = gd['is_real_libretto']
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                                    target, 
                                                                                    test_size = 0.3,
                                                                                    random_state=rstate)

In [ ]:
good_model.fit(features_train, target_train)

In [ ]:
analyze_model_classwise(good_model, features_test, target_test)

In [ ]:
importances = good_model.feature_importances_
forest_importances = pd.Series(importances, index=list_features_pyd)
std = np.std([tree.feature_importances_ for tree in good_model.estimators_], axis=0)

fig, ax = plt.subplots()
fig.set_size_inches(10, 6)
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
result = permutation_importance(
    good_model, features_test, target_test, n_repeats=10, random_state=rstate, n_jobs=2
)

In [ ]:
forest_importances = pd.Series(result.importances_mean, index=list_features_pyd)

In [ ]:
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
fig.set_size_inches(10, 6)
fig.tight_layout()
plt.show()

In [ ]:
assess_best_rf(gd, german_features, 'is_real_libretto')

In [ ]:
good_model = RandomForestClassifier(n_estimators = 600, random_state = rstate)

In [ ]:
features = standardize(gd, german_features)
target = gd['is_real_libretto']
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                                    target, 
                                                                                    test_size = 0.3,
                                                                                    random_state=rstate)

In [ ]:
good_model.fit(features, target)

In [ ]:
get_feature_importances(gd, good_model, german_features, 'German')

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
best = 0 

for i in range(1,20):
    model = DecisionTreeClassifier(max_depth=i, random_state=rstate)
    cross_val_scores = classify(model, gd, list_features_pyd)
    meanscore = np.mean(cross_val_scores)
    print(meanscore)
    if meanscore > best:
        best = meanscore
        best_depth = i
        best_tree_model = model

print(f'Лучший результат: {best} Его дала модель c глубиной {best_depth}')

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
best = 0 
for i in range(100,5000,100):
    model = LogisticRegression(max_iter=i, random_state=rstate)
    cross_val_scores = classify(model, gd, list_features_pyd)
    meanscore = np.mean(cross_val_scores)
    print(meanscore)
    if meanscore > best:
        best = meanscore
        best_max_iter = i
        best_log_reg_model = model

print(f'Лучший результат: {best} Его дала модель c {best_max_iter} итераций')

### Другие классификаторы

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
perceptron_model = MLPClassifier(random_state=rstate)

In [ ]:
result = classify(perceptron_model, gd, list_features_pyd)

In [ ]:
np.mean(result)

In [ ]:
plt.bar(list(range(len(result))), list(result))
ax = plt.gca()

#ax.set_xlim([xmin, xmax])
ax.set_ylim([0, 1])

### French classifier

In [ ]:
fd_lt['genre_with_libretto_subgenres'].unique()

In [ ]:
assess_best_brf(fd_lt, french_features, 'genre_with_libretto_subgenres')

In [ ]:
good_model = BalancedRandomForestClassifier(n_estimators = 250, random_state = rstate)

In [ ]:
features = standardize(fd_lt, french_features)
target = fd_lt['genre_with_libretto_subgenres']
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                                    target, 
                                                                                    test_size = 0.3,
                                                                                    random_state=rstate)

In [ ]:
good_model.fit(features_train, target_train)

In [ ]:
analyze_model_classwise(good_model, features_test, target_test)

In [ ]:
get_feature_importances(fd_lt, good_model, french_features, 'French')

### Binary

In [ ]:
assess_best_rf(fd, french_features, 'is_real_libretto')

In [ ]:
good_model = RandomForestClassifier(n_estimators = 750, random_state = rstate)

In [ ]:
features = standardize(fd, french_features)
target = fd['is_real_libretto']
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                                    target, 
                                                                                    test_size = 0.3,
                                                                                    random_state=rstate)

In [ ]:
good_model.fit(features_train, target_train)

In [ ]:
get_feature_importances(fd, good_model, french_features, 'French')

In [ ]:
get_feature_importances(fd, good_model, list_features_pyd)

In [ ]:
assess_best_rf(fd, list_features_pyd, 'is_real_libretto')

Trying 4-classes

In [ ]:
assess_best_rf(fd, french_features, 'libretto_or_genre')

In [ ]:
assess_best_rf(fd, list_features_pyd, 'libretto_or_genre')

## Subtitle exploration

In [ ]:
german_libretti = df_libretti_only[df_libretti_only['lang'] == 'ger']

In [ ]:

german_libretti['subtitle'].value_counts(ascending=True)[-15:].plot.barh()
#plt.xticks(rotation = 80)

In [ ]:
french_libretti = df_libretti_only[df_libretti_only['lang'] == 'fre']

In [ ]:
french_libretti['subtitle'].value_counts(ascending=True)[-15:].plot.barh()

In [ ]:
french_libretti[french_libretti['subtitle'].str.lower().str.contains('com')]['subtitle'].shape

In [ ]:
french_libretti.shape

In [ ]:
french_libretti[french_libretti['subtitle'].str.lower().str.contains('trag')]['subtitle'].shape

In [ ]:
german_libretti[(german_libretti['subtitle'].str.lower().str.contains('kom')) |
                (german_libretti['subtitle'].str.lower().str.contains('poss')) 
                ]['subtitle'].shape

In [ ]:
german_libretti['subtitle'].to_excel('subtitles_de.xls')

In [ ]:
french_libretti['subtitle'].to_excel('subtitles_fr.xls')

### Wordclouds

In [ ]:
from wordcloud import WordCloud

In [ ]:
import spacy

In [ ]:
! python -m spacy download de_core_news_sm

In [ ]:
german_nlp = spacy.load("de_core_news_sm")

In [ ]:
def lemmatize_string_de(string):
    doc = german_nlp(string)
    tokens = [token.lemma_ for token in doc] 
    return tokens

In [ ]:
lemmatize_string('Große romantische Oper in drei Akten')

In [ ]:
german_libretti['lemm_subtitle'] = german_libretti['subtitle'].apply(lemmatize_string)

In [ ]:
german_libretti['lemm_subtitle']

In [ ]:
from collections import Counter

In [ ]:
def get_flattened(words):
    flattened = []
    for somelist in words:
        for word in somelist:
            flattened.append(word)
    return flattened

In [ ]:
def produce_wordcloud(df, columnname, numwords):
    words = list(df[columnname])
    flattened = get_flattened(words)
    counted = dict(Counter(flattened).most_common(numwords))
    wordcloud = WordCloud(width=800, height=600).generate_from_frequencies(counted)
    plt.figure(figsize=(16,16))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'wordcloud')
    plt.axis("off")
    plt.savefig(f'wordcloud.jpg', dpi=300, facecolor='k', bbox_inches='tight')

In [ ]:
produce_wordcloud(german_libretti, 'lemm_subtitle', 50)

##  TODO: Dropping out the last act: what does it happen? Does the "comedification" of opera stop?

# TODO: Other peculiarities of libretti



* Prologues are often genre-signalling: is the relative frequence of `<div type="prologue">` higher in our libretti corpus?
* are operas inherently more action-packed than plays? check drama change rate
* computing ratio `<sp>` vs. `<stage>`: is opera more narrative than plays? --> already recognisable in the correlation matrix

* making sense of music inside operas: how does it influence the dramatic structure? e.g. checking relevant vocabulary in stage directions
    *   compare the lexical composition of stage directions between libretti and non-libretti

# III. Trash

## Older notes

### Observations 7.12:



*   We see a clear **non-libretti zone**: libretti have some requirements
*   To see which threshold values of which features are restrictive for libretti we can simply **plot pairwise boxplots** (see Szemes & Vida)
*   Libretti do not all cluster together, but we observe clustering of **some** libretti. The picture might be distorted due to overlapping of different time periods on one plot, need to check if splitting time  periods will make clusters clearer. 
*   Hypothesis to check: **later clusters might be more clear** because the genre of opera becomes more formalized ('Aristotelis of the opera emerges out of the initial diversity and unifies everyone') 



TODO: 

*   <s>Vectors for the features in LDA graphs -- checked, too many features, does not really help the interpretation</s>
*   <s>Split the true libretti and the putative libretti (libretti apartheid yay!)</s>
*   <s>Split timeframes</s>
*   Run only on thing which have 'Oper(a)' in the name
*   Rerun on French stuff



# Code snippets and trash


### Trash

### Removing unclassified plays

In [ ]:
filtered_data = german_data[german_data['libretto_or_genre'] != 'Other']

In [ ]:
filtered_data['genre_with_putative_libretto'].value_counts()

In [ ]:
filtered_data['color'].value_counts()

In [ ]:
x = filtered_data.loc[:, features].values
#x.apply()
# Separating out the target
#y = df.loc[:,['target']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)
X_LDA_fitted = LDA(n_components=3).fit(x, filtered_data['libretto_or_genre'])
X_LDA = X_LDA_fitted.transform(x)

In [ ]:
X_LDA

In [ ]:
plot_2d(filtered_data, X_LDA[:, 0],X_LDA[:, 1])

### Time-frames after apartheid (OLD)

#### 1770_1820

In [ ]:
german_data['year_normalized']

In [ ]:
german_data_1770_1820 = filtered_data[(filtered_data['year_normalized'] >= 1770) & 
                                      (filtered_data['year_normalized'] <= 1820)]

In [ ]:
x = german_data_1770_1820.loc[:, features].values
#x.apply()
# Separating out the target
#y = df.loc[:,['target']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)
X_LDA_fitted = LDA(n_components=2).fit(x, german_data_1770_1820['libretto_or_genre'])
X_LDA = X_LDA_fitted.transform(x)

In [ ]:
plot_2d(german_data_1770_1820, X_LDA[:, 0],X_LDA[:, 1])

#### 1821_1870

In [ ]:
german_data_1821_1870 = filtered_data[(filtered_data['year_normalized'] >= 1821) & 
                                      (filtered_data['year_normalized'] <= 1870)]

In [ ]:
german_data_1821_1870['genre_with_putative_libretto'].value_counts()

In [ ]:
x = german_data_1821_1870.loc[:, features].values
#x.apply()
# Separating out the target
#y = df.loc[:,['target']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)
X_LDA_fitted = LDA(n_components=2).fit(x, german_data_1821_1870['libretto_or_genre'])
X_LDA = X_LDA_fitted.transform(x)

In [ ]:
plot_2d(german_data_1821_1870, X_LDA[:, 0],X_LDA[:, 1])

In [ ]:
german_data_1871_1920 = filtered_data[(filtered_data['year_normalized'] >= 1871) & 
                                      (filtered_data['year_normalized'] <= 1920)]

In [ ]:
x = german_data_1871_1920.loc[:, features].values
#x.apply()
# Separating out the target
#y = df.loc[:,['target']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)
X_LDA_fitted = LDA(n_components=2).fit(x, german_data_1871_1920['libretto_or_genre'])
X_LDA = X_LDA_fitted.transform(x)

In [ ]:
plot_2d(german_data_1871_1920, X_LDA[:, 0],X_LDA[:, 1])

### French data

In [ ]:
fre_corpus = pydracor.Corpus('fre')

In [ ]:
french_data = fre_corpus.metadata()

In [ ]:
french_data = pd.DataFrame(french_data)

In [ ]:
french_data

In [ ]:
libretti_ids = metadata_df['id']

In [ ]:
metadata_df['is_ger'] = metadata_df['id'].str.contains('ger')

In [ ]:
french_data['is_real_libretto'] = french_data['id'].isin(libretti_ids)

In [ ]:
# how many libretti are there in the german part?
french_data['is_real_libretto'].sum()

In [ ]:
french_data['is_real_libretto']

In [ ]:
french_data['libretto_or_genre'] = french_data.apply(lambda x: x['normalized_genre'] if x['is_real_libretto'] == False else 'Libretto', axis=1)

In [ ]:
french_data['libretto_or_genre'].value_counts()

In [ ]:
french_data['libretto_or_genre'] = french_data['libretto_or_genre'].fillna('Other')

In [ ]:
french_data['libretto_or_genre'].value_counts()

In [ ]:
french_data['genre_with_putative_libretto'] = french_data.apply(lambda x: 'Libretto (attributed)' if 
                  x['is_real_libretto'] is True and x['libretto'] is False
                  else x['libretto_or_genre'], axis=1)

In [ ]:
french_data['genre_with_putative_libretto'].value_counts()

In [ ]:
french_data['genre_with_putative_libretto'] = french_data['genre_with_putative_libretto'].replace('Libretto', 'Libretto (DraCor)')

In [ ]:
french_data['genre_with_putative_libretto'].value_counts()

#### Remaking colors to add 6th category

In [ ]:
genres = list(french_data['genre_with_putative_libretto'].unique())

In [ ]:
len(genres)

In [ ]:
genres

In [ ]:
colors = ['blue', 'green', 'white',  'orange', 'red', 'yellow']

In [ ]:
genre_color_mapping = zip(genres, colors)

In [ ]:
genre_color_mapping = dict(genre_color_mapping)

In [ ]:
genre_color_mapping

In [ ]:
french_data['color'] = french_data['genre_with_putative_libretto'].apply(lambda x: 
                                                              genre_color_mapping[x])

### Removing unclassified plays

In [ ]:
filtered_french_data = french_data[french_data['libretto_or_genre'] != 'Other']

In [ ]:
filtered_french_data['genre_with_putative_libretto'].value_counts()

In [ ]:
filtered_french_data['color'].value_counts()

In [ ]:
filtered_french_data.info()

In [ ]:
filtered_french_data = filtered_french_data[~filtered_french_data['average_clustering'].isna()]

In [ ]:
x = filtered_french_data.loc[:, features].values
#x.apply()
# Separating out the target
#y = df.loc[:,['target']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)
X_LDA_fitted = LDA(n_components=3).fit(x, filtered_french_data['libretto_or_genre'])
X_LDA = X_LDA_fitted.transform(x)

In [ ]:
len(x)

In [ ]:
len(X_LDA)

In [ ]:
X_LDA

In [ ]:
plot_2d(filtered_french_data, X_LDA[:, 0],X_LDA[:, 1])

In [ ]:
import plotly.express as px

In [ ]:
df = px.data.iris()
features = list_features_pyd

fig = px.scatter_matrix(
    metadata_df,
    dimensions=features,
    color="libretto"
)
fig.update_traces(diagonal_visible=False)
fig.show()

* Broad analysis of mean and median for the values set in the variable *values_broad_analysis*. Look at list of column names to select different values. At the moment set to:

    * Number of Characters
    * Max Degree
    * Average Degree 
    * Density
    * Average Path Length
    * Average Clustering Coefficient

* Detailed analysis of mean and median by time frame (to be selected) set in the variable *values_detailed_analysis*. At the moment set to:
    * Network Size (number of characters in the play)
    * Density

In [ ]:
# set values for broad analysis
values_broad_analysis = ["numOfSpeakers", "maxDegree", "averageDegree", "density", "averagePathLength",
                          "averageClustering"]

In [ ]:
# set values for detailed analysis
values_detailed_analysis = ["size", "density"]

### 3. Perform Analysis: Investigate mean and median of values selected for broad analysis

##### Mean values

In [ ]:
metadata_genre_grouped[values_broad_analysis].mean()

##### Median values

In [ ]:
metadata_genre_grouped[values_broad_analysis].median()

### 4. Preparation: Exclude plays without genre information

In [ ]:
# delete rows with genre value "other"
metadata_df_genre_specified = metadata_df.drop(metadata_genre_grouped.get_group(other_val).index)
metadata_genre_specified_grouped = metadata_df_genre_specified.groupby([genre_key])

### 4. Genre specific analysis for values specified for detailed analysis

##### Mean values

In [ ]:
for key in values_detailed_analysis:
    metadata_genre_specified_grouped.mean()[key].plot(kind ="bar", subplots=True)
    plt.show()

In [ ]:
for key in values_detailed_analysis:
    metadata_genre_specified_grouped.mean()[key].plot(kind ="bar", subplots=True)
    plt.show()

##### Median values

In [ ]:
for key in values_detailed_analysis:
    metadata_genre_specified_grouped.median()[key].plot(kind ="bar", subplots=True)
    plt.show()

### 5. Time specific analysis

* interval size: set to the number of years you want one time interval to span, e.g. 30 (must be a number)
* threshold: Exclude time interval if it contains fewer texts than the thrseshold indicates

#### Get info about earliest and latest play

In [ ]:
year_key = "yearNormalized"
earliest = int(min(metadata_df_genre_specified[year_key]))
latest = int(max(metadata_df_genre_specified[year_key]))
print(f"Earliest play: {earliest}")
print(f"Latest play: {latest}")

#### Set time parameters for analysis

In [ ]:
while True:
    interval_size = input("Please enter the size of the time intervals (must be a number): ")
    if not interval_size.isnumeric():
        print("Your input is not valid. Please try again and enter a number.")
    else:
        interval_size = int(interval_size)
        print("Success!")
        break

In [ ]:
while True:
    threshold = input("Please enter the threshold (must be a number): ")
    if not threshold.isnumeric():
        print("Your input is not valid. Please try again and enter a number.")
    else:
        threshold = int(threshold)
        print("Success!")
        break

### Perform time specific analysis

In [ ]:
def round_down_to_ten(x):
        offset = x % 10
        return x - offset 
    
def get_time_periods(start, highest_range, period_length):
    time_periods = []
    start = round_down_to_ten(start)
    end = start + period_length
    while end < highest_range:
        time_periods.append((start, end))       
        start = end
        end = start + period_length
    time_periods.append((start,end))
    return time_periods

def get_time_period_fit(periods, year):
    for period in periods: 
        if year >= period[0] and year < period[1]:
            return f"{period[0]}-{period[1]}"
    if not math.isnan(year):
        print(f"No period found for year: {year}")
    return float("NaN")

##### Print time frames

In [ ]:
# create time frames according to user input
time_period_name = "timePeriod"
time_periods = get_time_periods(earliest, latest, interval_size)
time_periods

#### Split data into timeframes and filter by selected threshold

In [ ]:
# for each play, retrieve corresponding time frame
period_column = metadata_df_genre_specified[year_key].apply(lambda x: get_time_period_fit(time_periods, x))
metadata_df_genre_specified[time_period_name] = period_column

# apply threshold, if number of plays in one timeframe below the threshold -> exclude columns
metadata_df_time_genre_specified_filtered = metadata_df_genre_specified.groupby([time_period_name, genre_key]).filter(
lambda x: len(x) >= threshold)

# group data by genre and time frame
metadata_df_time_genre_grouped = metadata_df_time_genre_specified_filtered.groupby([time_period_name, genre_key])

#### Display number of plays that remain for each time frame after filtering

In [ ]:
metadata_df_time_genre_grouped.count()["name"]

### Plot development of genres 
* Median and mean values are calculated by time frame

##### Median values

In [ ]:
for key in values_detailed_analysis:
    print(key)
    metadata_df_time_genre_grouped[key].median().unstack().plot(figsize=(8,8)).legend(loc='upper left')
    plt.show()

##### Mean values

In [ ]:
for key in values_detailed_analysis:
    print(key)
    metadata_df_time_genre_grouped[key].mean().unstack().plot(figsize=(8,8)).legend(loc='upper left')
    plt.show()

### Display tabular 

##### Median values 

In [ ]:
for key in values_detailed_analysis:
    print(key)
    print(metadata_df_time_genre_grouped[key].median())
    print("\n")

##### Mean values 

In [ ]:
for key in values_detailed_analysis:
    print(key)
    print(metadata_df_time_genre_grouped[key].mean())
    print("\n")

# Trashbin below this point!

## Feature selection (older version deprecated)

### Features kept after correlation testing: 
* 'num_of_segments', 
* 'num_of_speakers', 
* 'num_of_person_groups',
* 'word_count_sp',
* 'word_count_stage', 
* 'average_degree', 
* 'density', 
* 'average_clustering',
* 'max_degree', 
* 'num_connected_components', 
* 'diameter',
* 'average_path_length'

### Original features list
* 'averageClustering', 
* 'density', 
* 'averagePathLength', 
* 'averageDegree', 
* 'diameter', 
* 'maxDegree', 
* 'numOfSpeakers', 
* 'numOfSpeakersFemale',
* 'numOfSpeakersMale', 
* 'numPersonGroups',
* 'numConnectedComponents', 
* 'numOfSegments' (try with and WITHOUT),
* 'wordCountText', 
* 'wordCountSp'/'wordCountStage'

In [ ]:
list_features_pyd = ['num_of_segments', 'num_of_speakers', 'num_of_person_groups',
                      'word_count_sp', 'word_count_stage', 'average_degree', 'density', 'average_clustering',
                      'max_degree', 'num_connected_components', 'diameter', 'average_path_length']

## Correlation matrix

In [ ]:
feature_correlations = metadata_df[list_features_pyd].corr()

In [ ]:
metadata_df = metadata_df.fillna('0')

In [ ]:
plt.matshow(feature_correlations)
plt.show()

In [ ]:
feature_correlations

In [ ]:
fig, ax = plt.subplots(figsize=(10,8)) 
seaborn.heatmap(feature_correlations, annot=True, cmap = 'coolwarm')

### Correlation matrix on both at once (experimental, to be removed):

In [ ]:
ger_french_data = pd.concat([german_data, french_data])

In [ ]:
ger_french_data.shape

In [ ]:
ger_french_data['num_connected_components'].value_counts()

In [ ]:
1905/2157

In [ ]:
feature_correlations = ger_french_data[list_features_pyd].corr()

In [ ]:
feature_correlations

In [ ]:
#metadata_df = metadata_df.fillna('0')

In [ ]:
plt.matshow(feature_correlations)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8)) 
seaborn.heatmap(feature_correlations, annot=True, cmap = 'coolwarm')
plt.savefig('correlation_matrix.png', dpi=300)

## Drop correlated features

In [ ]:
metadata_df = metadata_df.drop(['diameter', 'max_degree'], axis=1)

In [ ]:
list_features_pyd.remove('diameter')

In [ ]:
list_features_pyd.remove('max_degree')

## PCA on libretti only

### Creating year filter

We investigate three timeframes:

*   From xx to xx (only FRE)
*   From xx to xx (only FRE)
*   From xx to xx (FRE + GER)
*   From xx to xx (FRE + GER)
*   From xx to xx (FRE + GER)

In [ ]:
metadata_df

In [ ]:
metadata_df['year_normalized'].describe()

In [ ]:
metadata_df_1770_1820 = metadata_df[(metadata_df['year_normalized'] >= 1770) & 
                                      (metadata_df['year_normalized'] <= 1820)]

In [ ]:
metadata_df_1821_1870 = metadata_df[(metadata_df['year_normalized'] >= 1821) & 
                                      (metadata_df['year_normalized'] <= 1870)]

In [ ]:
metadata_df_1871_1920 = metadata_df[(metadata_df['year_normalized'] >= 1871) & 
                                      (metadata_df['year_normalized'] <= 1920)]

In [ ]:
metadata_df_1770_1820.shape

In [ ]:
df = metadata_df_1770_1820 

----

In [ ]:
df = df.reset_index()

In [ ]:
df['is_ger'] = df['id'].str.contains('ger')

In [ ]:
# ratio of german to french plays in this period
df.groupby('is_ger').count()['id']

In [ ]:
features = list_features_pyd
# Separating out the features
x = df.loc[:, features].values
# Separating out the target
#y = df.loc[:,['target']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

feature_vectors = pca.components_.T

In [ ]:
principalDf.shape

In [ ]:
df[['is_ger']].shape

In [ ]:
df['name'].shape

In [ ]:
finalDf = pd.concat([principalDf, df[['is_ger']], df['name']], axis = 1)#join="inner")

In [ ]:
finalDf.shape

In [ ]:
fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = [True, False]
colors = ['r', 'g', 'b']


    

for target, color in zip(targets,colors):
    indicesToKeep = finalDf['is_ger'] == target
    pc1 = finalDf.loc[indicesToKeep, 'principal component 1']
    pc2 = finalDf.loc[indicesToKeep, 'principal component 2']
    ax.scatter(pc1
               , pc2
               , c = color
               , s = 50)
    names = finalDf.loc[indicesToKeep, 'name']
    for i, txt in enumerate(list(names)):
        #print(i)
        #print(txt)
        #print(list(pc1)[i])
        ax.annotate(txt, (list(pc1)[i], list(pc2)[i])) 

    coeff = feature_vectors
    n = coeff.shape[0]
    labels = list_features_pyd
    for i in range(n):
        plt.arrow(0, 0, coeff[i,0], coeff[i,1],color = 'r',alpha = 0.5)
        if labels is None:
            plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, "Var"+str(i+1), color = 'g', ha = 'center', va = 'center')
        else:
            plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, labels[i], color = 'g', ha = 'center', va = 'center')
ax.legend(targets)
ax.grid()




*   Label dots -- Done
*   Ger vs Fre -- Done
*   Century snapshots -- Done
*   Add vectors -- Half-Done
*   Add reference corpora





In [ ]:
# which plays are the biggest?
df.sort_values('word_count_sp', ascending=False)[['name','word_count_sp']]

In [ ]:
#german_libretti = metadata_df[metadata_df['is_ger']]

In [ ]:
#french_libretti = metadata_df[~metadata_df['is_ger']]

In [ ]:
#french_libretti

In [ ]:
#german_libretti

In [ ]:
#german_libretti = german_libretti.reset_index()

In [ ]:
#french_libretti = french_libretti.reset_index(drop=True)

In [ ]:
#french_libretti

In [ ]:
#german_libretti

In [ ]:
def produce_PCA(df, target_column, features):
    #features = list_features_pyd
    # Separating out the features
    x = df.loc[:, features].values
    #x.apply()
    # Separating out the target
    #y = df.loc[:,['target']].values
    # Standardizing the features
    x = StandardScaler().fit_transform(x)

    #x = np.log(x)

    #print(x)


    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents
                , columns = ['principal component 1', 'principal component 2'])

    feature_vectors = pca.components_.T

    finalDf = pd.concat([principalDf, df[target_column], df['name']], axis = 1)#join="inner")

    fig = plt.figure(figsize = (30,30))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('Principal Component 1', fontsize = 12)
    ax.set_ylabel('Principal Component 2', fontsize = 12)
    ax.set_title('2 component PCA', fontsize = 10)
    
    targets = list(df[target_column].unique())
    
    colors = ['red', 'green', 'orange', 'grey', 'yellow'] 

    for target, color in zip(targets, colors):
        indicesToKeep = finalDf[target_column] == target
        pc1 = finalDf.loc[indicesToKeep, 'principal component 1']
        pc2 = finalDf.loc[indicesToKeep, 'principal component 2']
        ax.scatter(pc1
                , pc2
                , c = color
                , s = 150)
        names = finalDf.loc[indicesToKeep, 'name']
        #for i, txt in enumerate(list(names)):
            #print(i)
            #print(txt)
            #print(list(pc1)[i])
            #ax.annotate(txt, (list(pc1)[i], list(pc2)[i])) 

        coeff = feature_vectors
        n = coeff.shape[0]
        labels = features
        for i in range(n):
            plt.arrow(0, 0, coeff[i,0], coeff[i,1],color = 'r',alpha = 0.5)
            if labels is None:
                plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, "Var"+str(i+1), color = 'g', ha = 'center', va = 'center')
            else:
                plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, labels[i], color = 'g', ha = 'center', va = 'center')
    ax.legend(targets)
    ax.grid()

    #plt.xlim(10, 10) 
    #plt.ylim(10, 10)
    plt.savefig('pca.png')
    return x



In [ ]:
#produce_PCA(french_libretti, 'libretto', features)

1. Calculating genre distances

## Timesteps

In [ ]:
german_data['year_normalized'].describe()

In [ ]:
german_data_1770_1820 = german_data[(german_data['year_normalized'] >= 1770) & 
                                      (german_data['year_normalized'] <= 1820)]

In [ ]:
german_data_1770_1820[['name', 'year_normalized']]

In [ ]:
#german_data_1770_1820[german_data_1770_1820['name'].str.contains('journalisten')]

In [ ]:
german_data_1770_1820.to_csv('slice.csv')

In [ ]:
sliced = german_data_1770_1820.loc[:, features].values
#x.apply()
# Separating out the target
#y = df.loc[:,['target']].values
# Standardizing the features
sliced = StandardScaler().fit_transform(sliced)


In [ ]:
new_LDA = LDA(n_components=2).fit_transform(sliced, german_data_1770_1820['libretto_or_genre'])

In [ ]:
german_data_1770_1820

In [ ]:
plot_2d(german_data_1770_1820, new_LDA[:, 0],new_LDA[:, 1])

### Some PCA here

In [ ]:
import numpy as np

In [ ]:
x = produce_PCA(german_data, 'libretto_or_genre', list_features_pyd)

In [ ]:
pca_table = pd.DataFrame(x)

In [ ]:
pca_table[0].max()

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
len(x)

In [ ]:
x = german_data.loc[:, features].values
x = StandardScaler().fit_transform(x)

In [ ]:
labels = list(german_data['name'])

In [ ]:
#labels
name_to_color

In [ ]:
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
model = model.fit(x)
plt.title("Hierarchical Clustering Dendrogram")
plt.rcParams["figure.figsize"] = (30,30)
# plot the top three levels of the dendrogram
plot_dendrogram(model, labels=labels) #,link_color_func=lambda k: name_to_color[k])#, truncate_mode="level", p=6) 
plt.rcParams.update({'font.size': 24})

ax = plt.gca()
xlbls = ax.get_xmajorticklabels()
for lbl in xlbls:
    lbl.set_color(name_to_color[lbl.get_text()])

plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.savefig('histogram.png', dpi=300)
plt.show()

In [ ]:
genre_color_mapping

## Old LDA code

In [ ]:
def plot_2d(df, component1, component2):
    feature_vectors = []
    for i in range(10):
        new_feature_vector = go.Scatter(x = [0,scalings[i][0]],
                     y = [0,scalings[i][1]],
                     marker = dict(size = 1,
                                    color = "white"),
                     line = dict(color = "black",
                                width = 6),
                     name = features[i]
                     )
        feature_vectors.append(new_feature_vector)
    

    full_plot = go.Scatter(
        x = component1,
        y = component2,
        mode='markers',
        marker=dict(
            size=20,
            color=df['color'], #set color equal to a variable
            colorscale='Rainbow', # one of plotly colorscales
            showscale=True,
            line_width=1
        ),
        text=df['name']
    )
    feature_vectors.append(full_plot)
    fig = go.Figure(data=feature_vectors)
    fig.update_layout(margin=dict( l=100,r=100,b=100,t=100),width=2000,height=1200)                 
    fig.layout.template = 'plotly'
    
    fig.show()

In [ ]:
plot_2d(german_data, X_LDA[:, 0],X_LDA[:, 1])

In [ ]:
dict(size = 1, color = "rgb(84,48,5)")

In [ ]:
german_data.boxplot('num_connected_components')

In [ ]:
german_data[german_data['num_connected_components']>5]

In [ ]:
german_data[german_data['id'] == 'ger000413']

In [ ]:
scalings

In [ ]:
def plot_3d(component1,component2,component3,scalings,features):
    feature_vectors = []
    for i in range(10):
        new_feature_vector = go.Scatter3d(x = [0,scalings[i][0]],
                     y = [0,scalings[i][1]],
                     z = [0,scalings[i][2]],
                     marker = dict(size = 1,
                                    color = "rgb(84,48,5)"),
                     line = dict(color = "aquamarine",
                                width = 6),
                     name = features[i]
                     )
        feature_vectors.append(new_feature_vector)

    lda_plot = [go.Scatter3d(
        x=component1,
        y=component2,
        z=component3,
        mode='markers',
        text=german_data['name'],
        marker=dict(
            size=10,
            color=german_data['color'],                # set color to an array/list of desired values
            colorscale='Rainbow',   # choose a colorscale
            opacity=1,
            line_width=1
        )
    )
    ]
    lda_plot.extend(feature_vectors)
    fig = go.Figure(data=lda_plot)
# tight layout
    fig.update_layout(margin=dict(l=50,r=50,b=50,t=50),width=1800,height=1000)
    fig.layout.template = 'plotly_dark'


    fig.show()

In [ ]:
features

In [ ]:
plot_3d(X_LDA[:, 0],X_LDA[:, 1],X_LDA[:, 2], scalings, features)

In [ ]:
german_data.boxplot('num_of_segments')

In [ ]:
german_data[german_data['num_of_speakers'] > 200]

In [ ]:
german_data['num_of_segments'].median()

In [ ]:
german_data.describe()